# Creating the PainterPalette dataset from paintings datasets

The aim of this project is to create a dataset of painters from datasets such as WikiArt and Art500k, combining features, extending missing data of painters with web scraping through Google and Wiki API, and then creating links between painters based on similarity of style, geographical and social interaction.

One long-term goal would be to create a JSON file that contains all combined hierarchically. For example, a level in the structure could be art movement, inside it are artists with some base data like birthplace, year of birth and death and other geographical data, inside it are paintings with all contained data (even better would be including eras of painters in their substructure, and inside them the paintings). Then we could use this to create a network of art movements, artists, and paintings.

NEXT STEPS:<br>
- Combine more common authors from the two datasets (but under different name), and take in authors who are not in either datasets.<br>
- Remove non-painters from the datasets.<br>
- Finish and add Art500k artist dataset updates. <br>


FURTHER STEPS:<br>
- Turn the dataset into JSON format, and add pictures data

In [2]:
import pandas as pd
import numpy as np
import helper_functions

### WikiArt data

In [23]:
artists_wikiart = pd.read_csv('https://raw.githubusercontent.com/me9hanics/PainterPalette/main/datasets/wikiart_artists.csv')
artists_wikiart

,artist,styles,movement,styles_extended,pictures_count,birth_place,birth_year,death_year,death_place,gender,citizenship,occupations,locations,locations_with_years
0,Ad Reinhardt,"Abstract Art, Abstract Expressionism, Color Fi...",Abstract Expressionism,"{Abstract Art:15},{Abstract Expressionism:5},{...",52,Buffalo,1913.0,1967.0,New York City,male,United States of America,"painter, university teacher, printmaker, colla...",['New York City'],[]
1,Adnan Coker,"Abstract Art, Abstract Expressionism",Abstract Art,"{Abstract Art:25},{Abstract Expressionism:3}",28,NaN,1927.0,2022.0,NaN,male,NaN,painter,[],[]
2,Akkitham Narayanan,Abstract Art,Abstract Art,{Abstract Art:17},17,Kerala,1939.0,NaN,NaN,male,Dominion of India,"printmaker, painter",[],[]
3,Alberto Magnelli,"Abstract Art, Art Nouveau (Modern), Cubism, Ex...",Abstract Art,"{Abstract Art:19},{Art Nouveau (Modern):2},{Cu...",35,Florence,1888.0,1971.0,Meudon,male,Italy,"illustrator, painter","['Florence', 'Paris']",[]
4,Alekos Kontopoulos,"Abstract Art, Cubism, Expressionism, Post-Impr...",Social Realism,"{Abstract Art:26},{Cubism:5},{Expressionism:10...",79,Lamia,1904.0,1975.0,Athens,male,Greece,"writer, painter",[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3198,Serhij Schyschko,Unknown,Academic Art,{Unknown:9},9,Nosivka,1911.0,1997.0,Kyiv,male,Soviet Union,painter,[],[]
3199,Vudon Baklytsky,Unknown,Soviet Nonconformist Art,{Unknown:46},46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3200,Wolfgang Tillmans,Unknown,Contemporary,{Unknown:9},9,Remscheid,1968.0,NaN,NaN,male,Germany,"photographer, printmaker","['New York City', 'Berlin', 'London']",['New York City:1996-1996']
3201,Wu Daozi,Unknown,Tang Dynasty (618–907),{Unknown:8},8,Chang'an,680.0,740.0,Chang'an,male,Tang dynasty,painter,[],[]


Artists grouped by style data

In [5]:
wa_grouped = pd.read_csv('https://raw.githubusercontent.com/me9hanics/PainterPalette/main/datasets/wikiart_artists_styles_grouped.csv')
print("Length:", len(wa_grouped), "\n", "Number of groups with only 1 count:", len(wa_grouped[wa_grouped['count']==min(wa_grouped['count'])]))
wa_grouped[wa_grouped['artist'].str.contains("Monet")].sort_values(by=['count'], ascending=False)

Length: 7646 
 Number of groups with only 1 count: 1115


,style,artist,movement,count
2963,Impressionism,Claude Monet,Impressionism,1341
5468,Realism,Claude Monet,Impressionism,12
7041,Unknown,Claude Monet,Impressionism,12
462,Academicism,Claude Monet,Impressionism,1
3339,Japonism,Claude Monet,Impressionism,1


### Art500K

First dataset (from official website)

In [24]:
art500k_artists = pd.read_csv('https://raw.githubusercontent.com/me9hanics/PainterPalette/main/datasets/art500k_artists.csv')
art500k_artists[0:8]

,artist,Nationality,PaintingSchool,ArtMovement,Influencedby,Influencedon,Pupils,Teachers,FriendsandCoworkers,FirstYear,LastYear,PaintingsExhibitedAt,StylesYears,StylesCount,PaintingsExhibitedAtCount,Contemporary,Type
0,Gustave Courbet,French,NaN,{Realism:272},"Rembrandt,Caravaggio,Diego Velazquez,Peter Pau...","Edouard Manet,Claude Monet,Pierre-Auguste Reno...",NaN,NaN,NaN,1830.0,1877.0,"London, Montpellier, Moscow, CA, UK, Norway, D...","Realism:1835-1877,Romanticism:1830-1849","{Realism:257}, {Romanticism:13}","{France:88},{Switzerland:7},{Lille:8},{Paris:4...",No,Painting/Sculpture
1,Auguste Rodin,French,NaN,"{Modern art:3},{Impressionism:91}","Michelangelo,Donatello,","Georgia O'Keeffe,Man Ray,Aristide Maillol,Olex...","Constantin Brancusi,",NaN,NaN,1865.0,1985.0,"London, CA, UK, Switzerland, Lisbon, US, Germa...",Impressionism:1865-1905,{Impressionism:90},"{France:52},{Paris:15},{Brussels:2},{Belgium:1...",NaN,Painting/Sculpture
2,Frida Kahlo,Mexican,NaN,"{Naïve Art (Primitivism),Surrealism:99}","Amedeo Modigliani,Diego Rivera,Jose Clemente O...","Judy Chicago,Georgia O'Keeffe,Feminist Art,",NaN,NaN,NaN,1922.0,1954.0,"CA, LA, New York, US, New Orleans, Washington ...","Naïve Art (Primitivism):1922-1954,Surrealism:1...","{Naïve Art (Primitivism):99}, {Surrealism:15}","{Mexico:50},{San Francisco:6},{New York:4},{Me...",No,Painting/Sculpture
3,Banksy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011.0,2011.0,"Los Angeles, London, UK, Palestine, California...",NaN,NaN,"{Palestine:1},{Los Angeles:3},{California:3},{...",Yes,Painting/Sculpture
4,El Greco,"Spanish,Greek",Cretan School,"{Spanish Renaissance:1},{Renaissance:2},{Manne...","Byzantine Art,","Expressionism,Cubism,Eugene Delacroix,Edouard ...",NaN,"Titian,","Giulio Clovio,",1568.0,1614.0,"Seville, London, Illescas, Romania, Moscow, Gr...",Mannerism (Late Renaissance):1568-1600,"{Renaissance:2}, {XVI CenturySpanish Painting:...","{Spain:75},{Boston:1},{MA:1},{US:27},{Museo de...",No,Painting/Sculpture
5,Diego Rivera,Mexican,"Mexican Mural Renaissance,La Ruche","{Social Realism,Muralism:146}","Marc Chagall,Robert Delaunay,","Frida Kahlo,Pedro Coronel,Vlady,",NaN,NaN,"Amedeo Modigliani,Saturnino Herran,Roberto Mon...",1904.0,1956.0,"Moscow, CA, Acapulco, New York, Spain, Northam...","Cubism:1912-1916,Muralism:1922-1956,Art Deco:1...","{Post-impressionism:1}, {Cubism:19}, {Mexican ...","{France:1},{Paris:1},{Moscow:1},{Acapulco:2},{...",No,Painting/Sculpture
6,Claude Monet,French,NaN,"{Modern art:3},{Impressionism:1340}","Gustave Courbet,Charles-Francois Daubigny,John...","Childe Hassam,Robert Delaunay,Wassily Kandinsk...",NaN,"Eugene Boudin,Charles Gleyre,","Alfred Sisley,Pierre-Auguste Renoir,Camille Pi...",1858.0,1926.0,"London, Main, Moscow, Rotterdam, Giverny, CA, ...","Impressionist:1879-1904,Impressionism:1864-192...",{Nineteenth-Century European PaintingImpressio...,"{France:79},{Giverny:1},{London:6},{UK:15},{Bo...",No,Painting/Sculpture
7,Francisco Goya,Spanish,NaN,{Romanticism:391},"Albrecht Durer,Diego Velazquez,","Pablo Picasso,Chaim Soutine,Roberto Montenegro...",NaN,"José Luzán,Anton Raphael Mengs,",NaN,1760.0,1828.0,"London, Rotterdam, Museo del Prado, UK, Kunsth...",Romanticism:1760-1828,"{Neoclassicism / Portrait:1}, {Genre:1}, {Roma...","{Spain:168},{Paris:7},{Madrid:96},{Museo del P...",No,Painting/Sculpture


# Combining the two datasets

## TODO: Redo missing Art500k Wikidata name mapping

## TODO: Add styles (movements) from StylesCount, ArtMovements to "movements" field, cleanup

## 2024.10.27b) Finish, after having combined + modified instances in Art500k

In [7]:
import requests
import json

wikiart_artists = pd.read_csv('https://raw.githubusercontent.com/me9hanics/PainterPalette/main/datasets/wikiart_artists.csv')
art500k_artists = pd.read_csv('https://raw.githubusercontent.com/me9hanics/PainterPalette/main/datasets/art500k_artists.csv')
response = requests.get('https://raw.githubusercontent.com/me9hanics/PainterPalette/main/datasets/saves/painter_name_pairs.json')
wikiart_art500k_mapping = json.loads(response.text)
art500k_further_selected = pd.read_csv('https://raw.githubusercontent.com/me9hanics/PainterPalette/main/datasets/saves/art500k_further_selected.csv')
artists = helper_functions.create_painter_palette(wikiart_artists, art500k_artists, wikiart_art500k_mapping, art500k_further_selected)

In [4]:
artists.to_csv('datasets/artists.csv', index=False)
artists.to_csv('datasets/saves/artists_1_0_2.csv', index=False)
artists.to_csv('PainterPalette.csv', index=False)
artists.to_excel('PainterPalette.xlsx', index=False)

## 2024.10.27a): Combine instances that appear twice with same romanized lowercase name

See the important work below, just below this update.

In [3]:
import requests
import json
import unicodedata

def normalize_name(name):
    #Lowercase + replace non-ASCII characters
    name = name.lower()
    name = ''.join(c for c in unicodedata.normalize('NFKD', name) if c.isascii())
    return name

wikiart_artists = pd.read_csv('https://raw.githubusercontent.com/me9hanics/PainterPalette/main/datasets/wikiart_artists.csv')
art500k_artists = pd.read_csv('https://raw.githubusercontent.com/me9hanics/PainterPalette/main/datasets/art500k_artists.csv')
response = requests.get('https://raw.githubusercontent.com/me9hanics/PainterPalette/main/datasets/saves/painter_name_pairs.json')
response.encoding = 'utf-8'
wikiart_art500k_mapping = json.loads(response.text)
art500k_further_selected = pd.read_csv('https://raw.githubusercontent.com/me9hanics/PainterPalette/main/datasets/saves/art500k_further_selected.csv')
artists = helper_functions.create_painter_palette(wikiart_artists, art500k_artists, wikiart_art500k_mapping, art500k_further_selected)

#artists = pd.read_csv('https://raw.githubusercontent.com/me9hanics/PainterPalette/main/datasets/artists.csv')
artist_names = artists['artist'].to_numpy()
artist_index = np.array(artists.index)
duplicate_romanized_names = ["Jozsef Rippl-Ronai","Laszlo Moholy-Nagy","Jean-Leon Gerome","Constantin Brâncuși","Fernand Leger","Frantisek Kupka",
              "Josef Sima","Salvador Dali","Stanisław Ignacy Witkiewicz","Bui Xuan Phai","Joan Miro","Oyvind Fahlstrom","Rodolfo Arico",
              "Briton Riviere","Leon Bonnat","Mihaly Munkacsy","Miklos Barabas","Pedro Américo","Antoni Tapies","Emile Galle","Felix Vallotton",
              "Jules Cheret","Józef Mehoffer","Leon Bakst","Stanisław Wyspiański","Stefan Luchian","Wilhelm Trübner","Andre Masson",
              "Bartolome Esteban Murillo","Diego Velazquez","Gabriel Metsu","Nikolaus Knüpfer","Sebastien Bourdon","Edouard Vuillard","Emile Bernard",
              "Paul Serusier","Helio Oiticica","Francois Morellet","Andre Derain","Carlo Carra","Carlos Merida","Giacomo Manzu","Julio Gonzalez",
              "Paul Cezanne","Andrzej Wróblewski","Antonio Carneiro","Jose Gutierrez Solana","Jose Pancetti","Kathe Kollwitz","Laszlo Mednyanszky",
              "Moise Kisling","Oswaldo Guayasamin","Bartolome Bermejo","Eugene Boudin","Frederic Bazille","Honore Daumier","Jacques-Émile Blanche",
              "Jose Malhoa","Julian Fałat","Józef Pankiewicz","Nicolae Darascu","Peder Severin Kroyer","Santiago Rusinol","Vilhelm Hammershoi",
              "Vilhelms Purvitis","Jesus Rafael Soto","Jose Guadalupe Posada","Manuel Rodríguez Lozano","Theodore Chasseriau","Matthias Grünewald",
              "Eugene Delacroix","Théodore Géricault","Gertrude Kasebier","Charles-Francois Daubigny","Gustave Dore","Paja Jovanovic","Theodore Rousseau",
              "Theodule Ribot","Francois Boucher","José Campeche","Arnold Böcklin","Piotr Michałowski","Jef Aerosol","Albin Brunovsky","Alejandro Obregon",
              "Meret Oppenheim","Eugene Carriere","Felicien Rops","Leonid Sejka","Itō Jakuchū","Tani Bunchō","Paul César Helleu","Jean Dessès","Martín Rico",
              "Antônio Parreiras","Hishida Shunsō","Félix Bonfils","Ivan Generalić","Uemura Shōen","Charles François Daubigny",#"Luis Paret y Alcázar"
              "Nicolas Régnier","Pedro Rodriguez","Felix Lecomte","Kveta Pacovska","Louis Rémy Mignot","Pedro Weingärtner","Cecilio Plá"]

normalized_to_original = {}
duplicate_index_pairs = {}
for i in range(len(artist_names)):
    name = artist_names[i]
    normalized_name = normalize_name(name)
    index = artist_index[i]
    
    if normalized_name in normalized_to_original:
        normalized_to_original[normalized_name].append(name)
        duplicate_index_pairs[normalized_name].append(index)
    else:
        normalized_to_original[normalized_name] = [name]
        duplicate_index_pairs[normalized_name] = [index]

duplicates = {k: v for k, v in normalized_to_original.items() if len(v) > 1}
duplicate_index_pairs = {k: v for k, v in duplicate_index_pairs.items() if len(v) > 1}

We can see that after combining the Art500k instances (+ one WikiArt instance combination), we only have very few (4) cases of duplications, as these now arise only between the two datasets. We can just manually deal with these.

In [10]:
duplicates

{'georges de la tour': ['Georges de la Tour', 'Georges De La Tour'],
 'jozsef rippl-ronai': ['Jozsef Rippl-Ronai', 'József Rippl-Rónai'],
 'laszlo moholy-nagy': ['Laszlo Moholy-Nagy', 'László Moholy-Nagy'],
 'jean-leon gerome': ['Jean-Leon Gerome', 'Jean-Léon Gérôme']}

All of these duplicates actually have a pair in *painter_name_pairs* (`wikiart_art500k_mapping`).<br>
At this step, I combine instances further in the Art500k dataset, and reset the *painter_name_pairs* here in 3 cases.

In [21]:
wikiart_art500k_mapping['Jean-Leon Gerome'] = 'Jean-Léon Gérôme'
wikiart_art500k_mapping['Jozsef Rippl-Ronai'] = 'József Rippl-Rónai'
wikiart_art500k_mapping['Laszlo Moholy-Nagy'] = 'László Moholy-Nagy'

with open('datasets/saves/painter_name_pairs.json', 'w', encoding='utf-8') as f:
    json.dump(wikiart_art500k_mapping, f, ensure_ascii=False, indent=4)

## *Important work* 2024.10.25-27: Find duplicate instances that appear twice with same romanized lowercase name (also for Art500k and WikiArt)

These were found during an import of the dataset into SQL, as that is case insensitive and with the original configuration turns non-English characters into their closest English equivalent.<br>
This came in handy as it allowed to find duplicates not yet found - usually cases with multiple letter differences (both foreign letters and often capitalization).

This is the basis also for Art500k and WikiArt combinations, done in the separate notebooks.

In [3]:
import requests
import json
import unicodedata

def normalize_name(name):
    #Lowercase + replace non-ASCII characters
    name = name.lower()
    name = ''.join(c for c in unicodedata.normalize('NFKD', name) if c.isascii())
    return name

wikiart_artists = pd.read_csv('https://raw.githubusercontent.com/me9hanics/PainterPalette/main/datasets/wikiart_artists.csv')
art500k_artists = pd.read_csv('https://raw.githubusercontent.com/me9hanics/PainterPalette/main/datasets/art500k_artists.csv')
response = requests.get('https://raw.githubusercontent.com/me9hanics/PainterPalette/main/datasets/saves/painter_name_pairs.json')
response.encoding = 'utf-8'
wikiart_art500k_mapping = json.loads(response.text)
art500k_further_selected = pd.read_csv('https://raw.githubusercontent.com/me9hanics/PainterPalette/main/datasets/saves/art500k_further_selected.csv')
artists = helper_functions.create_painter_palette(wikiart_artists, art500k_artists, wikiart_art500k_mapping, art500k_further_selected)

#artists = pd.read_csv('https://raw.githubusercontent.com/me9hanics/PainterPalette/main/datasets/artists.csv')
artist_names = artists['artist'].to_numpy()
artist_index = np.array(artists.index)
duplicate_romanized_names = ["Jozsef Rippl-Ronai","Laszlo Moholy-Nagy","Jean-Leon Gerome","Constantin Brâncuși","Fernand Leger","Frantisek Kupka",
              "Josef Sima","Salvador Dali","Stanisław Ignacy Witkiewicz","Bui Xuan Phai","Joan Miro","Oyvind Fahlstrom","Rodolfo Arico",
              "Briton Riviere","Leon Bonnat","Mihaly Munkacsy","Miklos Barabas","Pedro Américo","Antoni Tapies","Emile Galle","Felix Vallotton",
              "Jules Cheret","Józef Mehoffer","Leon Bakst","Stanisław Wyspiański","Stefan Luchian","Wilhelm Trübner","Andre Masson",
              "Bartolome Esteban Murillo","Diego Velazquez","Gabriel Metsu","Nikolaus Knüpfer","Sebastien Bourdon","Edouard Vuillard","Emile Bernard",
              "Paul Serusier","Helio Oiticica","Francois Morellet","Andre Derain","Carlo Carra","Carlos Merida","Giacomo Manzu","Julio Gonzalez",
              "Paul Cezanne","Andrzej Wróblewski","Antonio Carneiro","Jose Gutierrez Solana","Jose Pancetti","Kathe Kollwitz","Laszlo Mednyanszky",
              "Moise Kisling","Oswaldo Guayasamin","Bartolome Bermejo","Eugene Boudin","Frederic Bazille","Honore Daumier","Jacques-Émile Blanche",
              "Jose Malhoa","Julian Fałat","Józef Pankiewicz","Nicolae Darascu","Peder Severin Kroyer","Santiago Rusinol","Vilhelm Hammershoi",
              "Vilhelms Purvitis","Jesus Rafael Soto","Jose Guadalupe Posada","Manuel Rodríguez Lozano","Theodore Chasseriau","Matthias Grünewald",
              "Eugene Delacroix","Théodore Géricault","Gertrude Kasebier","Charles-Francois Daubigny","Gustave Dore","Paja Jovanovic","Theodore Rousseau",
              "Theodule Ribot","Francois Boucher","José Campeche","Arnold Böcklin","Piotr Michałowski","Jef Aerosol","Albin Brunovsky","Alejandro Obregon",
              "Meret Oppenheim","Eugene Carriere","Felicien Rops","Leonid Sejka","Itō Jakuchū","Tani Bunchō","Paul César Helleu","Jean Dessès","Martín Rico",
              "Antônio Parreiras","Hishida Shunsō","Félix Bonfils","Ivan Generalić","Uemura Shōen","Charles François Daubigny",#"Luis Paret y Alcázar"
              "Nicolas Régnier","Pedro Rodriguez","Felix Lecomte","Kveta Pacovska","Louis Rémy Mignot","Pedro Weingärtner","Cecilio Plá"]

normalized_to_original = {}
duplicate_index_pairs = {}
for i in range(len(artist_names)):
    name = artist_names[i]
    normalized_name = normalize_name(name)
    index = artist_index[i]
    
    if normalized_name in normalized_to_original:
        normalized_to_original[normalized_name].append(name)
        duplicate_index_pairs[normalized_name].append(index)
    else:
        normalized_to_original[normalized_name] = [name]
        duplicate_index_pairs[normalized_name] = [index]

duplicates = {k: v for k, v in normalized_to_original.items() if len(v) > 1}
duplicate_index_pairs = {k: v for k, v in duplicate_index_pairs.items() if len(v) > 1}

Important information: no duplicate groups have more than 2 instances. (This is after two 3-instance Art500k rows were combined into one-one.)

In [26]:
#For each duplicate group (they all have length two), check the two names and check whether they are in WikiArt or Art500k or both.
#Based on that, put pairs of painter names into groups: wiki-wiki, wiki-art500k, art500k-art500k

from itertools import product

wiki_art500k_names_intra = []
wiki_wiki_pairs = []
wiki_art500k_pairs = []
art500k_art500k_pairs = []
wiki_names = wikiart_artists['artist'].to_numpy()
art500k_names = art500k_artists['artist'].to_numpy()

instances_dict = {}
for key in duplicates:
    name1 = duplicates[key][0]
    name2 = duplicates[key][1]

    instances_dict[name1] = []
    instances_dict[name2] = []
    if name1 in wiki_names:
        index1 = wikiart_artists[wikiart_artists['artist'] == name1].index[0]
        instances_dict[name1].append(("wiki", index1))
    if name1 in art500k_names:
        index2 = art500k_artists[art500k_artists['artist'] == name1].index[0]
        instances_dict[name1].append(("art500k", index2))
    if name2 in wiki_names:
        index1 = wikiart_artists[wikiart_artists['artist'] == name2].index[0]
        instances_dict[name2].append(("wiki", index1))
    if name2 in art500k_names:
        index2 = art500k_artists[art500k_artists['artist'] == name2].index[0]
        instances_dict[name2].append(("art500k", index2))
    
    #We need all the inbetween combinations: Cartesian product of the two sets
    for (type1, index1), (type2, index2) in product(instances_dict[name1], instances_dict[name2]):
        if type1 == "wiki" and type2 == "wiki":
            wiki_wiki_pairs.append((name1, name2))
        elif type1 == "wiki" and type2 == "art500k":
            wiki_art500k_pairs.append((name1, name2))
        elif type1 == "art500k" and type2 == "wiki":
            wiki_art500k_pairs.append((name1, name2))
        elif type1 == "art500k" and type2 == "art500k":
            art500k_art500k_pairs.append((name1, name2))

    #We also need the intra-group pairs (can only be art-wiki pairs)
    if len(instances_dict[name1]) == 2:
        wiki_art500k_names_intra.append(name1)
    if len(instances_dict[name2]) == 2:
        wiki_art500k_names_intra.append(name2)

We now make changes in the Art500k and WikiArt datasets to advance forward.

In [27]:
len(wiki_wiki_pairs), len(wiki_art500k_pairs), len(art500k_art500k_pairs), len(wiki_art500k_names_intra), wiki_wiki_pairs

(1, 140, 158, 136, [('Leonid Sejka', 'Leonid Šejka')])

The duplicates that SQL found but our program didn't also need to be investigated.

In [ ]:
unfound_duplicates = [k for k in duplicate_romanized_names if normalize_name(k) not in duplicates]
unfound_duplicates

## 2024.10.20 (Artists 1.0.2) Combine duplicates based on lowercase name

This was a temporary fix, more combinations are needed and we need to update the mappings.

In [3]:
import requests
import json

wikiart_artists = pd.read_csv('https://raw.githubusercontent.com/me9hanics/PainterPalette/main/datasets/wikiart_artists.csv')
art500k_artists = pd.read_csv('https://raw.githubusercontent.com/me9hanics/PainterPalette/main/datasets/art500k_artists.csv')
response = requests.get('https://raw.githubusercontent.com/me9hanics/PainterPalette/main/datasets/saves/painter_name_pairs.json')
wikiart_art500k_mapping = json.loads(response.text);
art500k_further_selected = pd.read_csv('https://raw.githubusercontent.com/me9hanics/PainterPalette/main/datasets/saves/art500k_further_selected.csv')
artists = helper_functions.create_painter_palette(wikiart_artists, art500k_artists, wikiart_art500k_mapping, art500k_further_selected)

#Get all name duplicates when ignoring capitalization
duplicates = artists.copy()
duplicates['artist_lowercase'] = duplicates['artist'].str.lower()
duplicates = duplicates[duplicates.duplicated(subset=['artist_lowercase'], keep=False)]
duplicates = duplicates.sort_values(by=['artist_lowercase'])
duplicates[:6]

,artist,Nationality,citizenship,gender,styles,movement,Art500k_Movements,birth_place,death_place,birth_year,...,PaintingSchool,Influencedby,Influencedon,Pupils,Teachers,FriendsandCoworkers,Contemporary,ArtMovement,Type,artist_lowercase
2047,Abraham van Strij,NaN,Netherlands,male,Neoclassicism,Neoclassicism,NaN,Dordrecht,Dordrecht,1753.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,abraham van strij
10347,Abraham Van Strij,Dutch,Netherlands,male,NaN,NaN,NaN,Dordrecht,Dordrecht,1753.0,...,NaN,NaN,NaN,NaN,NaN,NaN,No,{Neoclassicism:38},NaN,abraham van strij
10372,Adam Van Noort,Flemish,Southern Netherlands,male,NaN,NaN,NaN,Antwerp,Antwerp,1562.0,...,Antwerp School,"Jacob Jordaens,",NaN,"Peter Paul Rubens,Jacob Jordaens,",NaN,"Maarten de Vos,",No,"{Baroque,Mannerism (Late Renaissance):14}",NaN,adam van noort
855,Adam van Noort,NaN,Southern Netherlands,male,"Baroque, Mannerism (Late Renaissance), Norther...",Baroque,NaN,Antwerp,Antwerp,1562.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,adam van noort
2177,Adolph de Meyer,NaN,France,male,Pictorialism,Pictorialism,NaN,Paris,Los Angeles,1868.0,...,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,adolph de meyer
10301,Adolph De Meyer,"French,American",France,male,NaN,NaN,NaN,Paris,Los Angeles,1868.0,...,NaN,NaN,NaN,NaN,NaN,NaN,No,{Pictorialism:18},NaN,adolph de meyer


In [5]:
#Group by lowercase artist names for list of instances with each name
grouped_indices = duplicates.groupby('artist_lowercase').apply(lambda x: x.index.tolist())

for artist_lowercase, indices in grouped_indices.items():
    if len(indices) > 1:
        combined_index = indices[0]
        for i in range(1, len(indices)):
            next_index = indices[i]
            artists, combined_index = helper_functions.painter_palette_combine_instances_by_index(artists, combined_index, next_index, return_index=True)
artists = artists.reset_index(drop=True)

C:\Users\hanic\AppData\Local\Temp\ipykernel_19348\336713003.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_indices = duplicates.groupby('artist_lowercase').apply(lambda x: x.index.tolist())


In [9]:
artists.to_csv('datasets/artists.csv', index=False)
artists.to_csv('datasets/saves/artists_1_0_2.csv', index=False)
artists.to_csv('PainterPalette.csv', index=False)
artists.to_excel('PainterPalette.xlsx', index=False)

## 2024.05.07: Update mapping, re-run PainterPalette creating (adding WikiArt update)

Note: On 2024.07.13 Excel file saving was added.

In [3]:
import requests
import json

wikiart_artists = pd.read_csv('https://raw.githubusercontent.com/me9hanics/PainterPalette/main/datasets/wikiart_artists.csv')
art500k_artists = pd.read_csv('https://raw.githubusercontent.com/me9hanics/PainterPalette/main/datasets/art500k_artists.csv')
response = requests.get('https://raw.githubusercontent.com/me9hanics/PainterPalette/main/datasets/saves/painter_name_pairs.json')
wikiart_art500k_mapping = json.loads(response.text);
art500k_further_selected = pd.read_csv('https://raw.githubusercontent.com/me9hanics/PainterPalette/main/datasets/saves/art500k_further_selected.csv')

In [4]:
wikiart_art500k_mapping['Felix Nadar'] = 'Nadar'
wikiart_art500k_mapping_keys = list(wikiart_art500k_mapping.keys()); wikiart_art500k_mapping_values = list(wikiart_art500k_mapping.values())

In [6]:
artists = helper_functions.create_painter_palette(wikiart_artists, art500k_artists, wikiart_art500k_mapping, art500k_further_selected)

artists.to_csv('datasets/artists.csv', index=False)
artists.to_csv('datasets/saves/artists_1_0_1.csv', index=False)
artists.to_csv('PainterPalette.csv', index=False)
artists.to_excel('PainterPalette.xlsx', index=False)
with open('datasets/saves/painter_name_pairs.json', 'w', encoding='utf-8') as f:
    json.dump(wikiart_art500k_mapping, f, ensure_ascii=False, indent=4)

## 2024.03.29: Re-run PainterPalette creating (Art500k parts were significantly modified)

In [2]:
import requests
import json

wikiart_artists = pd.read_csv('https://raw.githubusercontent.com/me9hanics/PainterPalette/main/datasets/wikiart_artists.csv')
art500k_artists = pd.read_csv('https://raw.githubusercontent.com/me9hanics/PainterPalette/main/datasets/art500k_artists.csv')
response = requests.get('https://raw.githubusercontent.com/me9hanics/PainterPalette/main/datasets/saves/painter_name_pairs.json')
wikiart_art500k_mapping = json.loads(response.text); wikiart_art500k_mapping_keys = list(wikiart_art500k_mapping.keys()); wikiart_art500k_mapping_values = list(wikiart_art500k_mapping.values())
art500k_further_selected = pd.read_csv('https://raw.githubusercontent.com/me9hanics/PainterPalette/main/datasets/saves/art500k_further_selected.csv')

In [5]:
artists = helper_functions.create_painter_palette(wikiart_artists, art500k_artists, wikiart_art500k_mapping, art500k_further_selected)

In [8]:
artists.to_csv('datasets/artists.csv', index=False)
artists.to_csv('datasets/saves/artists_1_0.csv', index=False)
artists.to_csv('PainterPalette.csv', index=False)

## Version 1.0: 2024.03.19 (plus tries before) - Put the data combining into a function 

also: Rename *artist.csv* to *painter_palette.csv* in the base folder

In [2]:
import requests
import json

wikiart_artists = pd.read_csv('https://raw.githubusercontent.com/me9hanics/PainterPalette/main/datasets/wikiart_artists.csv')
art500k_artists = pd.read_csv('https://raw.githubusercontent.com/me9hanics/PainterPalette/main/datasets/art500k_artists.csv')
response = requests.get('https://raw.githubusercontent.com/me9hanics/PainterPalette/main/datasets/saves/painter_name_pairs.json')
wikiart_art500k_mapping = json.loads(response.text); wikiart_art500k_mapping_keys = list(wikiart_art500k_mapping.keys()); wikiart_art500k_mapping_values = list(wikiart_art500k_mapping.values())
art500k_further_selected = pd.read_csv('https://raw.githubusercontent.com/me9hanics/PainterPalette/main/datasets/saves/art500k_further_selected.csv')

In [3]:
artists = helper_functions.create_painter_palette(wikiart_artists, art500k_artists, wikiart_art500k_mapping, art500k_further_selected)

In [4]:
artists.to_csv('datasets/artists.csv', index=False)
artists.to_csv('datasets/saves/artists_1_0.csv', index=False)
artists.to_csv('PainterPalette.csv', index=False)

## 2024.03.12-18 - Get Wikidata item IDs for each selected Art500k artist

In [3]:
import requests
import json
import httpimport

with httpimport.remote_repo('https://raw.githubusercontent.com/me9hanics/sparql-wikidata-data-collection/main/'):
    import functions as external_functions

In [28]:
with open('datasets/saves/art500k_wikidata_names_mapping.json', 'r', encoding='utf-8') as f:
    art500k_wikidata_names_mapping = json.load(f)

In [5]:
art500k_wikidata_IDs_mapping = {}
unprocessed_names = []
for artist in art500k_wikidata_names_mapping.keys():
    artist_wikidata_id = external_functions.get_person_wikidata_id(artist)
    if artist_wikidata_id:
        art500k_wikidata_IDs_mapping[artist] = artist_wikidata_id
    else:
        unprocessed_names.append(artist)
        art500k_wikidata_IDs_mapping[artist] = None

KeyboardInterrupt: 

After some time, the run was manually interrupted, and the data was saved to a file. <br>
The data was then loaded and the process later was continued.

This was repeated multiple times.

In [25]:
with open('datasets/saves/art500k_wikidata_IDs_mapping.json', 'w', encoding='utf-8') as f:
    json.dump(art500k_wikidata_IDs_mapping, f, ensure_ascii=False, indent=4)

In [16]:
with open('datasets/saves/temporary/art500k_wikidata_unprocessed_names.json', 'w', encoding='utf-8') as f:
    json.dump(unprocessed_names, f, ensure_ascii=False, indent=4)

Reload:

In [6]:
with open('datasets/saves/art500k_wikidata_IDs_mapping.json', 'r', encoding='utf-8') as f:
    art500k_wikidata_IDs_mapping_pt1 = json.load(f)

with open('datasets/saves/temporary/art500k_wikidata_unprocessed_names.json', 'r', encoding='utf-8') as f:
    unprocessed_names_pt1 = json.load(f)


In [13]:
art500k_wikidata_IDs_mapping = art500k_wikidata_IDs_mapping_pt1.copy()
unprocessed_names = unprocessed_names_pt1.copy()

In [30]:
not_yet_processed = [key for key in art500k_wikidata_names_mapping.keys() if key not in art500k_wikidata_IDs_mapping.keys()]

In [34]:
counter = 0
for artist in not_yet_processed:
    if counter % 1000 == 999:
        with open('datasets/saves/art500k_wikidata_IDs_mapping_pt2.json', 'w', encoding='utf-8') as f:
            json.dump(art500k_wikidata_IDs_mapping, f, ensure_ascii=False, indent=4)
        with open('datasets/saves/temporary/art500k_wikidata_unprocessed_names_pt2.json', 'w', encoding='utf-8') as f:
            json.dump(unprocessed_names, f, ensure_ascii=False, indent=4)
    artist_wikidata_id = external_functions.get_person_wikidata_id(artist)
    if artist_wikidata_id:
        art500k_wikidata_IDs_mapping[artist] = artist_wikidata_id
    else:
        unprocessed_names.append(artist)
        art500k_wikidata_IDs_mapping[artist] = None
    counter += 1

In [36]:
len(art500k_wikidata_IDs_mapping), len(unprocessed_names)

(10154, 4922)

Since we are done:

In [ ]:
with open('datasets/saves/art500k_wikidata_IDs_mapping.json', 'w', encoding='utf-8') as f:
    json.dump(art500k_wikidata_IDs_mapping, f, ensure_ascii=False, indent=4)

with open('datasets/saves/temporary/art500k_wikidata_unprocessed_names.json', 'w', encoding='utf-8') as f:
    json.dump(unprocessed_names, f, ensure_ascii=False, indent=4)

For unprocessed painters, their name in this format isn't found in the Wikidata database. An example would be the painter Constantin Brâncuși, who is not found by the name "Constantin Brancusi". (Actually, if we don't limit to human instances, a picture named after him is found, so it's important to limit the scope.)

For these cases, best is if we look into *painter_names_200k.txt* and see which names are closest to the ones we have, and manually check if they are the same person. If they are, we can get the Wikidata item ID from the "new name".

In [38]:
len(art500k_wikidata_IDs_mapping), len(unprocessed_names)

(10154, 4922)

In [39]:
#Read line by line the file painter_names_200k.txt

painter_names_url = 'https://raw.githubusercontent.com/me9hanics/PainterPalette/main/painter_names_200k.txt'
wiki_painter_names_200k = requests.get(painter_names_url).text.splitlines()

In [40]:
same_name_painters = np.intersect1d(unprocessed_names, wiki_painter_names_200k)
no_match_painters = np.setdiff1d(unprocessed_names, wiki_painter_names_200k)
len(same_name_painters), len(no_match_painters)

(1225, 3697)

In [41]:
same_name_painters

array(['A. Lutz', 'Abbas Al-Musavi', 'Adam Brincken', ...,
       'Édouard de Beaumont', 'Étienne-Simon Martin', 'Štefan Bubán'],
      dtype='<U81')

In [54]:
external_functions.get_all_person_info('A. Lutz')

{'name': 'A. Lutz',
 'birth_place': None,
 'birth_date': None,
 'death_date': None,
 'death_place': None,
 'gender': 'male',
 'citizenship': 'Kingdom of the Netherlands',
 'occupation': ['printmaker', 'etcher', 'painter', 'engraver', 'researcher'],
 'work_locations': [{'location': 'The Hague',
   'start_time': '1809-01-01T00:00:00Z',
   'end_time': '1822-01-01T00:00:00Z',
   'point_in_time': None}]}

We can get some data of these artists, even if not the Wikidata ID.<br>
That will be done in the above update.

In [53]:
with open('datasets/saves/temporary/painters_match_wikidata_name_no_ID.json', 'w', encoding='utf-8') as f:
    json.dump(list(same_name_painters), f, ensure_ascii=False, indent=4)

As for those artists who did not have a match, we check which names they are "closest to" among the 200k collected Wikidata painters. (We don't just have painters in our dataset, so some may not have a match.)

In [48]:
def calculate_similarities(artists_X, artists_Y):
    sims_df = pd.DataFrame(columns=['artist (original)','"Best" pair','Similarity', 'Character difference', 'Operations to transform'])
    artists_X_copy = artists_X.copy()
    artists_Y_copy = artists_Y.copy()
    if type(artists_X) == pd.core.frame.DataFrame:
        artists_X_copy = artists_X_copy['artist']
    if type(artists_Y) == pd.core.frame.DataFrame:
            artists_Y_copy = artists_Y_copy['artist']

    for painter in artists_X:
        all_sims = []
        max_sim = 0
        for comparison_artist in artists_Y:
            (similarity_score,char_diff, operation_count) = helper_functions.similarity_character_difference_operations(painter, comparison_artist)
            if similarity_score >= max_sim: #Runtime reasons
                max_sim = similarity_score
                all_sims.append((similarity_score,char_diff, operation_count ,comparison_artist))
        final_maximum = max(sims[0] for sims in all_sims) 
        for sims in all_sims:
            if sims[0] == final_maximum: #Just take the highest ones
                sims_df = pd.concat([sims_df, pd.DataFrame([[painter, sims[3], sims[0], sims[1], sims[2]]], columns=['artist (original)','"Best" pair','Similarity', 'Character difference', 'Operations to transform'])])

    sims_df = (sims_df.sort_values(by=['Similarity'], ascending=False)).reset_index(drop=True)
    return sims_df

In [62]:
similarities = calculate_similarities(no_match_painters, wiki_painter_names_200k)

C:\Users\hanic\AppData\Local\Temp\ipykernel_1856\4042882262.py:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  sims_df = pd.concat([sims_df, pd.DataFrame([[painter, sims[3], sims[0], sims[1], sims[2]]], columns=['artist (original)','"Best" pair','Similarity', 'Character difference', 'Operations to transform'])])


In [189]:
similarities

,artist (original),"""Best"" pair",Similarity,Character difference,Operations to transform
0,Tethart Philip Christian Haag,Tethart Philipp Christian Haag,0.983051,1,1
1,Frederick Marlett Bell-Smith,Frederic Marlett Bell-Smith,0.981818,1,1
2,Cornelis van Poelenburgh,Cornelius van Poelenburgh,0.979592,1,1
3,Hans Leonard Schäufelein,Hans Leonhard Schäufelein,0.979592,1,1
4,Francis Leggatt Chantrey,Francis Legatt Chantrey,0.978723,1,1
...,...,...,...,...,...
4939,MP5,Maria Pop,0.333333,6,2
4940,MP5,Mel Prest,0.333333,4,2
4941,MP5,Márta Pán,0.333333,6,2
4942,MP5,MWF,0.333333,2,1


In [64]:
similarities.to_csv('datasets/saves/temporary/similarities_no_match_painters.csv', index=False)

In [196]:
yet_to_check_df = similarities.copy()
matches_df = (similarities[(similarities['Similarity'] > 0.85) & (similarities['Character difference'] == 1 ) ]).copy()
no_matches_df = (similarities[(similarities['Similarity'] < 0.5) & (similarities['Character difference'] > 1 ) ]).copy()
repeat_df = pd.DataFrame(columns=['artist (original)','"Best" pair','Similarity', 'Character difference', 'Operations to transform'])

In [197]:
yet_to_check_df = yet_to_check_df.drop(matches_df.index).drop(no_matches_df.index)

In [104]:
def move_instances_from_A_to_B(A, B, rows_A):
    A_ = A.copy()
    B_ = B.copy()
    for index, row in rows_A.iterrows():
        A_ = A_[A_['artist (original)'] != row['artist (original)']]
        B_ = pd.concat([B_, row.to_frame().T])
    return A_, B_

def move_instances_from_A_to_B_by_index(A, B, indexes):
    A_ = A.copy()
    B_ = B.copy()
    for index in indexes:
        row = A_.loc[index]
        A_ = A_.drop(index)
        B_ = pd.concat([B_, row.to_frame().T])
    return A_, B_

### Rounds:

#### "1st round":

In [198]:
move_to_skip = [143, 227, 352, 354, 355, 457,458,460,481, 510,511,512,513, 522, 544, 550, 557,559, 561, 567,568,570,572, 612,673,728,729]
move_to_repeat_A = [64, 67, 75, 102, 136, 137, 226, 232, 562, 579, 593, 598, 605,666, 676, 681, 684, 688, 700, 714, 715, 717,
       719, 721, 722, 723, 730, 732, 739, 747, 751]

In [199]:
matches_df, repeat_df = move_instances_from_A_to_B_by_index(matches_df,repeat_df, move_to_repeat_A)
matches_df, no_matches_df = move_instances_from_A_to_B_by_index(matches_df, no_matches_df, move_to_skip)

#### 2nd round:

In [242]:
no_matches_df = pd.concat([no_matches_df, yet_to_check_df[yet_to_check_df['Similarity'] < 0.6]])
yet_to_check_df = yet_to_check_df[yet_to_check_df['Similarity'] >= 0.6]

In [243]:
no_matches_df = pd.concat([no_matches_df,yet_to_check_df[(yet_to_check_df['Operations to transform']== 1 ) & (yet_to_check_df['Similarity'] <0.74 )]])
yet_to_check_df = yet_to_check_df[~yet_to_check_df.index.isin(no_matches_df.index)] #Easier to copy

In [244]:
move_to_match = [4734, 4281, 3561, 3343, 3187, 3130, 3156, 3159, 3114, 2919, 2739, 2578, 3168]
move_to_retry = [4188, 3237, 3015, 2513, 2523, 2549, 2510]

In [255]:
no_matches_df, repeat_df = move_instances_from_A_to_B_by_index(no_matches_df, repeat_df, move_to_retry)
no_matches_df, matches_df = move_instances_from_A_to_B_by_index(no_matches_df, matches_df, move_to_match)

In [256]:
no_matches_df, matches_df = move_instances_from_A_to_B_by_index(no_matches_df, matches_df)

#### 3rd round:

In [285]:
yet_to_check_df, no_matches_df=move_instances_from_A_to_B_by_index(yet_to_check_df, no_matches_df, list(yet_to_check_df[(yet_to_check_df['Operations to transform'] >5 ) ].index) )

In [287]:
yet_to_check_df, no_matches_df=move_instances_from_A_to_B_by_index(yet_to_check_df, no_matches_df,[3167] )

In [306]:
yet_to_check_df, no_matches_df=move_instances_from_A_to_B_by_index(yet_to_check_df, no_matches_df, list(yet_to_check_df[(yet_to_check_df['Operations to transform'] ==5 ) ].index) )
yet_to_check_df, no_matches_df=move_instances_from_A_to_B_by_index(yet_to_check_df, no_matches_df, list(yet_to_check_df[(yet_to_check_df['Operations to transform'] ==4 ) ].index) )

In [307]:
move_to_match = [2080, 538]

no_matches_df, matches_df = move_instances_from_A_to_B_by_index(no_matches_df, matches_df, move_to_match)

In [308]:
matches_df.to_csv('datasets/saves/temporary/matches_df.csv', index=False)
no_matches_df.to_csv('datasets/saves/temporary/no_matches_df.csv', index=False)
repeat_df.to_csv('datasets/saves/temporary/repeat_df.csv', index=False)
yet_to_check_df.to_csv('datasets/saves/temporary/yet_to_check_df.csv', index=False)

#### Round 4:

In [339]:
yet_to_check_df, no_matches_df = move_instances_from_A_to_B_by_index(yet_to_check_df, no_matches_df, list(yet_to_check_df[(yet_to_check_df['artist (original)']).str.len() < 8].index) )
yet_to_check_df, repeat_df = move_instances_from_A_to_B_by_index(yet_to_check_df, repeat_df, list(yet_to_check_df[(yet_to_check_df['Operations to transform'] ==3 ) ].index))

In [364]:
move_to_match = [3099, 1884]
move_to_match_B = [186, 197, 312, 450, 655]
move_to_skip_B = [1114, 4681]

In [366]:
repeat_df, no_matches_df = move_instances_from_A_to_B_by_index(repeat_df, no_matches_df, move_to_skip_B)
repeat_df, matches_df =move_instances_from_A_to_B_by_index(repeat_df, matches_df, move_to_match_B)
no_matches_df, matches_df = move_instances_from_A_to_B_by_index(no_matches_df, matches_df, move_to_match)

#### Round 5:

In [376]:
yet_to_check_df, matches_df = move_instances_from_A_to_B_by_index(yet_to_check_df, matches_df, list(yet_to_check_df[(yet_to_check_df['Operations to transform'] == 2 ) & (yet_to_check_df['Similarity'] > 0.904 ) ].index) )

In [481]:
move_to_match = [40,69, 71, 82, 113, 126, 130, 146, 153, 166, 172, 196, 265, 275, 325, 326, 336, 340, 345, 351, 366,373,392,393, 396, 397, 4575]
move_to_skip = [219, 272,  305, 314, 401]; move_to_skip_B = [220,240,245, 268,303]
move_to_retry = [   301,310,311,315, 319, 339, 370, 379,  390, 391,394,395, 398,399]
move_to_retry_B = [234, 283, 293,300, 313,324,330,333,341,342, 353, 364, 372,387,]

In [478]:
matches_df, repeat_df = move_instances_from_A_to_B_by_index(matches_df, repeat_df, move_to_retry_B)
yet_to_check_df, repeat_df = move_instances_from_A_to_B_by_index(yet_to_check_df, repeat_df, move_to_retry)
yet_to_check_df, no_matches_df = move_instances_from_A_to_B_by_index(yet_to_check_df, no_matches_df, move_to_skip)
matches_df, no_matches_df = move_instances_from_A_to_B_by_index(matches_df, no_matches_df, move_to_skip_B)
yet_to_check_df, matches_df = move_instances_from_A_to_B_by_index(yet_to_check_df, matches_df, move_to_match)

#### Round 6:

In [522]:
move_to_match = [404, 409, 412, 413, 414, 418, 422, 424,425, 427, 444,445,447,448, 459,479, 487, 491, 493,499,500, 502,503, 629, 654, 656, 657, 2053,]
move_to_match_B = [2061,2063, 2077,2127,2129,2203,2206, 2316, 2337,2353,2436, 2437,2472, 3107]
move_to_skip = [426]

In [505]:
yet_to_check_df, repeat_df = move_instances_from_A_to_B_by_index(yet_to_check_df, repeat_df, list(yet_to_check_df[(yet_to_check_df['Similarity'] < 0.767 ) ].index))

In [526]:
yet_to_check_df, matches_df = move_instances_from_A_to_B_by_index(yet_to_check_df, matches_df, move_to_match)
repeat_df, matches_df = move_instances_from_A_to_B_by_index(repeat_df, matches_df, move_to_match_B)
yet_to_check_df, no_matches_df = move_instances_from_A_to_B_by_index(yet_to_check_df, no_matches_df, move_to_skip)

In [530]:
_, repeat_df = move_instances_from_A_to_B_by_index(yet_to_check_df, repeat_df, list(yet_to_check_df.index))

Save last groups:

In [531]:
matches_df.to_csv('datasets/saves/temporary/matches_df.csv', index=False)
no_matches_df.to_csv('datasets/saves/temporary/no_matches_df.csv', index=False)
repeat_df.to_csv('datasets/saves/temporary/repeat_df.csv', index=False)

Yet to check dataframe was deleted.

### Search for more IDs:

In [8]:
import json
matches = pd.read_csv('datasets/saves/temporary/matches_df.csv')
#no_matches = pd.read_csv('datasets/saves/temporary/no_matches_df.csv')
repeat = pd.read_csv('datasets/saves/temporary/repeat_df.csv')
repeat = list(repeat['artist (original)'].unique())
with open('datasets/saves/art500k_wikidata_IDs_mapping.json', 'r', encoding='utf-8') as f:
    art500k_wikidata_IDs_mapping1 = json.load(f)

In [13]:
for index, row in matches.iterrows():
    artist_original = row['artist (original)']
    artist_name_wikidata = row['"Best" pair']
    wikidata = external_functions.get_all_person_info_improved(artist_name_wikidata)
    if wikidata:
        if 'id' in wikidata.keys():
            art500k_wikidata_IDs_mapping1[artist_original] = wikidata['id']
            continue
    print("No Wikidata ID for: ", artist_original) #This only happens if one of the if statements above is not met

No Wikidata ID for:  Nishida Shun'ei


In [15]:
for artist in repeat:
    wikidata = external_functions.get_all_person_info_improved(artist)
    if wikidata:
        if 'id' in wikidata.keys():
            art500k_wikidata_IDs_mapping1[artist] = wikidata['id']
            continue
    print("No wikidata for", artist)

No wikidata for Dei
No wikidata for Jansson
No wikidata for Wia
No wikidata for Zoid
No wikidata for Federico de Madrazo
No wikidata for Pieter Coecke
No wikidata for Cornelius & Co.
No wikidata for Cornelisse
No wikidata for Alexandrovsky
No wikidata for Mangbetu
No wikidata for Wiener Werkstätte
No wikidata for Simonetta et Fabiani
No wikidata for Stevens & Williams
No wikidata for Los Four
No wikidata for Hill & Adamson
No wikidata for German School
No wikidata for Dominick & Haff
No wikidata for The Blind
No wikidata for Gilbert & George
No wikidata for Mabel Early
No wikidata for Reed & Barton
No wikidata for La Courtille
No wikidata for Bread and Butter
No wikidata for Bodanzky
No wikidata for Keystone
No wikidata for Sukenaga
No wikidata for Anne Frank House
No wikidata for Val Saint Lambert
No wikidata for Buechner
No wikidata for History San José
No wikidata for Grantham
No wikidata for Longines
No wikidata for Stavrakis
No wikidata for Monvoisin
No wikidata for Marimekko
No w

Manual additions:

In [19]:
art500k_wikidata_IDs_mapping1["Nishida Shun'ei"] = 'Q7040526'
art500k_wikidata_IDs_mapping1["Federico de Madrazo"] = 'Q1345186'
art500k_wikidata_IDs_mapping1["Pieter Coecke"] = 'Q434006'
art500k_wikidata_IDs_mapping1["Francesco di Gentile da Fabriano"] = 'Q312393'
art500k_wikidata_IDs_mapping1["Louis-Nicolas"] = 'Q26844700'
art500k_wikidata_IDs_mapping1["Captain William Baillie"] = 'Q8004950'
art500k_wikidata_IDs_mapping1["Manuel Capdevila"] = 'Q9027691'
art500k_wikidata_IDs_mapping1["Constantin Huygens"] = 'Q560746'
art500k_wikidata_IDs_mapping1["Yun Hyong Keun"] = 'Q73668649'
art500k_wikidata_IDs_mapping1["Hendrick van Balen"] = 'Q642071'

In [21]:
with open('datasets/saves/art500k_wikidata_IDs_mapping.json', 'w', encoding='utf-8') as f:
    json.dump(art500k_wikidata_IDs_mapping1, f, ensure_ascii=False, indent=4)

## 2024.03.12 - Minor addition to the mapping

In [22]:
import requests
import json

response = requests.get('https://raw.githubusercontent.com/me9hanics/PainterPalette/main/datasets/saves/painter_name_pairs.json')
wikiart_art500k_mapping = json.loads(response.text); wikiart_art500k_mapping_keys = list(wikiart_art500k_mapping.keys()); wikiart_art500k_mapping_values = list(wikiart_art500k_mapping.values())

In [27]:
wikiart_art500k_mapping['José Pedro Costigliolo'] = 'Costigliolo'

In [28]:
with open('datasets/saves/painter_name_pairs.json', 'w', encoding='utf-8') as f:
    json.dump(wikiart_art500k_mapping, f, ensure_ascii=False)

## 2024.03.09-10 (-11): Gather Art500k artists that are not in the WikiArt dataset, but are associated with a Wikidata profile (new mapping)

### Gathering the valid Wikidata instance artists from Art500k:

(in some cases, this may make things less accurate)

In [2]:
import requests
import json
import httpimport

with httpimport.remote_repo('https://raw.githubusercontent.com/me9hanics/sparql-wikidata-data-collection/main/'):
    import functions as external_functions

art500k_artists = pd.read_csv("https://raw.githubusercontent.com/me9hanics/PainterPalette/main/datasets/art500k_artists.csv")
artists_wikiart = pd.read_csv('https://raw.githubusercontent.com/me9hanics/PainterPalette/main/datasets/wikiart_artists.csv')
response = requests.get('https://raw.githubusercontent.com/me9hanics/PainterPalette/main/datasets/saves/painter_name_pairs.json')
wikiart_art500k_mapping = json.loads(response.text); wikiart_art500k_mapping_keys = list(wikiart_art500k_mapping.keys()); wikiart_art500k_mapping_values = list(wikiart_art500k_mapping.values())

unprocessed_art500k_painters = art500k_artists[art500k_artists['artist'].isin(wikiart_art500k_mapping_values) == False]

In [29]:
art500k_wikidata_names_mapping = {}

In [30]:
for painter in unprocessed_art500k_painters['artist']:
    painter_returned_name = external_functions.get_person_wikidata_name(painter, delay=121)
    if painter_returned_name:
        art500k_wikidata_names_mapping[painter] = painter_returned_name

KeyboardInterrupt: 

In [ ]:
with open('datasets/saves/art500k_wikidata_names_mapping.json', 'w', encoding='utf-8') as f:
    json.dump(art500k_wikidata_names_mapping, f, ensure_ascii=False)

After some time of running, this was manually stopped, and a faster algorithm was used:

In [5]:
with open('datasets/saves/art500k_wikidata_names_mapping.json', 'r', encoding='utf-8') as f:
    art500k_wikidata_names_mapping_part1 = json.load(f)
len(art500k_wikidata_names_mapping_part1)

5057

In [7]:
unprocessed_art500k_painters = art500k_artists[art500k_artists['artist'].isin(wikiart_art500k_mapping_values) == False]
unprocessed_art500k_painters = unprocessed_art500k_painters[unprocessed_art500k_painters['artist'].isin(art500k_wikidata_names_mapping_part1.keys()) == False]
len(unprocessed_art500k_painters)

13194

In [8]:
art500k_wikidata_names_mapping_part_2 = {}
not_found = []
for painter in unprocessed_art500k_painters['artist']:
    painter_returned_name = external_functions.get_person_wikidata_name_fast(painter)
    if painter_returned_name:
        art500k_wikidata_names_mapping_part_2[painter] = painter_returned_name
    else:
        not_found.append(painter)

In [9]:
with open('datasets/saves/art500k_wikidata_names_mapping_part_2.json', 'w', encoding='utf-8') as f:
    json.dump(art500k_wikidata_names_mapping_part_2, f, ensure_ascii=False)

In [14]:
art500k_wikidata_names_mapping =art500k_wikidata_names_mapping_part1.copy()
art500k_wikidata_names_mapping.update(art500k_wikidata_names_mapping_part_2)

In [18]:
with open('datasets/saves/art500k_wikidata_names_mapping.json', 'w', encoding='utf-8') as f:
    json.dump(art500k_wikidata_names_mapping, f, ensure_ascii=False)

### Fetching the data of these artists from Wikidata:

In *art500k_updates.ipynb* (creating Art500k_selected)

## Version 2024.02.20 (0.3.): Same as before, minor column name change

In [6]:
import json
import requests

response = requests.get('https://raw.githubusercontent.com/me9hanics/PainterPalette/main/datasets/saves/painter_name_pairs.json')
mapping = json.loads(response.text); mapping_keys = list(mapping.keys())
artists_c = helper_functions.create_painter_dataset_from_mapping(mapping) #Function was changed from before
artists_wikiart = pd.read_csv('https://raw.githubusercontent.com/me9hanics/PainterPalette/main/datasets/wikiart_artists.csv')
additional_artists = artists_wikiart[~(artists_wikiart['artist'].isin(mapping_keys))]
additional_artists.rename(columns={'pictures_count':'wikiart_pictures_count'}, inplace=True)
artists = pd.concat([artists_c, additional_artists], ignore_index=True)

C:\Users\hanic\AppData\Local\Temp\ipykernel_15972\3375940034.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  additional_artists.rename(columns={'pictures_count':'wikiart_pictures_count'}, inplace=True)


In [9]:
artists[50:60]

,artist,Nationality,citizenship,gender,styles,movement,Art500k_Movements,birth_place,death_place,birth_year,...,occupations,PaintingsExhibitedAt,PaintingsExhibitedAtCount,PaintingSchool,Influencedby,Influencedon,Pupils,Teachers,FriendsandCoworkers,Contemporary
50,Richard Pousette-Dart,American,United States of America,male,"Abstract Art, Abstract Expressionism, Academicism",Abstract Art,{Abstract Expressionism:54},Saint Paul,Rockland County,1916.0,...,"photographer, painter, drawer","NY, New York City, US","{New York City:2},{NY:2},{US:2}","New York School,Irascibles",NaN,NaN,NaN,NaN,NaN,NaN
51,Ethel Léontine Gabain,"French,British",United Kingdom,female,Neo-Romanticism,Neo-Romanticism,NaN,Le Havre,London,1883.0,...,"lithographer, painter","London, Manchester, UK","{London:2},{UK:3},{Manchester:1}",NaN,NaN,NaN,NaN,NaN,NaN,No
52,Charles-Amable Lenoir,NaN,France,male,"Academicism, Unknown",Academic Art,{Academic Art:9},Châtelaillon-Plage,Paris,1860.0,...,painter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53,Francisco de Zurbaran,Spanish,Spain,male,"Baroque, Unknown",Baroque,{Baroque:96},Fuente de Cantos,Madrid,1598.0,...,painter,"Hungary, Museo del Prado, Paris, Barcelona, B...","{Grenoble:7},{France:19},{Seville:31},{Spain:3...",NaN,"Caravaggio,","Gustave Courbet,",NaN,"Francisco Pacheco,",NaN,No
54,Pieter van Hanselaere,Belgian,Belgium,male,Neoclassicism,Neoclassicism,{Neoclassicism:8},Ghent,Ghent,1786.0,...,painter,"Netherlands, Amsterdam","{Amsterdam:2},{Netherlands:2}",NaN,NaN,NaN,NaN,"Jacques-Louis David,",NaN,No
55,Jean-Honore Fragonard,French,France,male,"Rococo, Unknown",Rococo,"{Rococo:72},{Renaissance:1}",Grasse,Paris,1732.0,...,"illustrator, painter, printmaker, architectura...","Netherlands, Paris,London, Pasadena, Moscow, ...","{France:21},{Paris:8},{Moscow:1},{Russia:3},{S...",NaN,NaN,NaN,NaN,NaN,NaN,No
56,Ion Theodorescu-Sion,Romanian,Romania,male,"Art Nouveau (Modern), Impressionism, Post-Impr...",Post-Impressionism,{Post-Impressionism:43},Ianca,Bucharest,1882.0,...,"trade unionist, caricaturist, painter",NaN,NaN,Balchik School,NaN,NaN,NaN,NaN,NaN,No
57,Janos Mattis-Teutsch,"Hungarian,Romanian",Romania,male,"Abstract Art, Constructivism, Cubism, Expressi...",Constructivism,"{Art Nouveau:1},{Socialist realism:1},{Abstrac...",Brașov,Brașov,1884.0,...,"writer, poet, painter, sculptor, journalist",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58,Apollinary Goravsky,"Belarusian,Russian",Russian Empire,male,Romanticism,Romanticism,{Romanticism:12},Novyja Nabarki,Mariinskaya Hospital,1833.0,...,painter,"Russia, Moscow, Saint Petersburg, Minsk, Belarus","{Minsk:7},{Belarus:7},{Saint Petersburg:2},{Ru...",NaN,"Belarusian National Museum of Fine Arts, Minsk...",NaN,NaN,NaN,NaN,No
59,Edouard Debat-Ponsan,French,France,male,Academicism,Academic Art,"{Academic art:1},{Academic Art:11}",Toulouse,Paris,1847.0,...,painter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No


In [10]:
artists.to_csv('datasets/artists.csv', index=False)
artists.to_csv('datasets/saves/artists_0_3.csv', index=False)
artists.to_csv('artists.csv', index=False)

## Version 2024.02.15 (0.3.): Same as before, modifications in columns

In [14]:
import json
import requests

response = requests.get('https://raw.githubusercontent.com/me9hanics/PainterPalette/main/datasets/saves/painter_name_pairs.json')
mapping = json.loads(response.text); mapping_keys = list(mapping.keys())
artists_c = helper_functions.create_painter_dataset_from_mapping(mapping) #Function was changed from before
artists_wikiart = pd.read_csv('https://raw.githubusercontent.com/me9hanics/PainterPalette/main/datasets/wikiart_artists.csv')
additional_artists = artists_wikiart[~(artists_wikiart['artist'].isin(mapping_keys))]
artists = pd.concat([artists_c, additional_artists], ignore_index=True)

In [16]:
artists.to_csv('datasets/artists.csv', index=False)
artists.to_csv('datasets/saves/artists_0_3.csv', index=False)
artists.to_csv('artists.csv', index=False)

## Version 2024.01.29 (0.2.) : Use the mapping to combine info from the two datasets, and take all other artists from WikiArt

In [45]:
import json
import requests

response = requests.get('https://raw.githubusercontent.com/me9hanics/PainterPalette/main/datasets/saves/painter_name_pairs.json')
mapping = json.loads(response.text); mapping_keys = list(mapping.keys())
artists_c = helper_functions.create_painter_dataset_from_mapping(mapping) #OLD VERSION OF THE FUNCTION
artists_wikiart = pd.read_csv('https://raw.githubusercontent.com/me9hanics/PainterPalette/main/datasets/wikiart_artists.csv')
#artists = pd.read_csv('https://raw.githubusercontent.com/me9hanics/PainterPalette/main/datasets/artists.csv')
additional_artists = artists_wikiart[~(artists_wikiart['artist'].isin(mapping_keys))]
artists = pd.concat([artists_c, additional_artists], ignore_index=True)

In [ ]:
artists.to_csv('datasets/artists.csv', index=False)
artists.to_csv('datasets/saves/artists_0_2.csv', index=False)
artists.to_csv('artists.csv', index=False)

## Update 2024.01.28 - Create a separate function to combine the two datasets by a mapping of artist names from one set to the other + create the previous update's mapping

Previous update's mapping saved as part of the previous update.<br>
Added function to *helper_functions.py*.

In [5]:
import json
import requests
from helper_functions import create_painter_dataset_from_mapping

response = requests.get('https://raw.githubusercontent.com/me9hanics/PainterPalette/main/datasets/saves/painter_name_pairs.json')
mapping = json.loads(response.text)

artists_c = create_painter_dataset_from_mapping(mapping)
artists_c

,artist,Nationality,birth_place,birth_year,styles,styles_extended,StylesYears,StylesCount,PlacesCount,Contemporary,...,FirstYear,LastYear,Places,PlacesYears,PaintingSchool,Influencedby,Influencedon,Pupils,Teachers,FriendsandCoworkers
0,Bracha L. Ettinger,"French,Jewish,Israeli",Tel Aviv,1948.0,New European Painting,{New European Painting:21},New European Painting:1991-2009,{New European Painting:21},NaN,Yes,...,1991.0,2009.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,William H. Johnson,American,Florence,1901.0,"Cubism, Expressionism, Futurism, Naïve Art (Pr...","{Cubism:1},{Expressionism:24},{Futurism:1},{Na...","Naïve Art (Primitivism):1938-1946,Expressionis...","{Naïve Art (Primitivism):74}, {Expressionism:4...",NaN,No,...,1923.0,1946.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Alexey Bogolyubov,Russian,NaN,NaN,"Realism, Romanticism","{Realism:25},{Romanticism:19}","Realism:1860-1889,Romanticism:1845-1860","{Realism:25}, {Romanticism:19}","{Rybinsk:2},{Russia:9},{Saint Petersburg:6},{M...",No,...,1845.0,1889.0,"Saint Petersburg, Rybinsk, Russia, Moscow","Rybinsk:1879-1889,Russia:1850-1889,Saint Peter...",Peredvizhniki (Society for Traveling Art Exhib...,NaN,NaN,NaN,NaN,NaN
3,O. Louis Guglielmi,"American,Egyptian",Cairo,1906.0,"Cubism, Expressionism, Magic Realism","{Cubism:3},{Expressionism:6},{Magic Realism:25}","Magic Realism:1931-1946,Cubism:1946-1954,Expre...","{Magic Realism:25}, {Cubism:8}, {Expressionism:6}",NaN,No,...,1931.0,1955.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Mikalojus Konstantinas Ciurlionis,NaN,Varėna,1875.0,Symbolism,{Symbolism:168},NaN,NaN,NaN,No,...,1905.0,1905.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2649,Marianne von Werefkin,NaN,Tula,1860.0,Unknown,{Unknown:61},NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2650,Robert Demachy,French,Saint-Germain-en-Laye,1859.0,Unknown,{Unknown:24},NaN,NaN,{France:2},No,...,1900.0,1914.0,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2651,Wolfgang Tillmans,NaN,Remscheid,1968.0,Unknown,{Unknown:9},NaN,NaN,"{London:1},{United Kingdom:1}",Yes,...,2001.0,2001.0,"London, United Kingdom",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2652,Wu Daozi,Chinese,Chang'an,680.0,Unknown,{Unknown:8},NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Four fathers of Chinese painting,NaN,NaN,NaN,NaN,NaN


Missing cases:

In [9]:
artists = pd.read_csv('https://raw.githubusercontent.com/me9hanics/PainterPalette/main/datasets/artists.csv')
artists[~(artists['artist'].isin(list(artists_c['artist'])))]

,artist,Nationality,birth_place,birth_year,styles,styles_extended,StylesYears,StylesCount,PlacesCount,Contemporary,...,FirstYear,LastYear,Places,PlacesYears,PaintingSchool,Influencedby,Influencedon,Pupils,Teachers,FriendsandCoworkers
2458,John Frederick Herring Sr.,NaN,NaN,NaN,Romanticism,{Romanticism:79},NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2459,Willem van Swanenburg,NaN,Leiden,1580.0,Baroque,{Baroque:18},NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2460,Leon Battista Alberti,NaN,Genoa,1404.0,Early Renaissance,{Early Renaissance:7},NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2465,Giuseppe Barberis,NaN,Turin,1517.0,Romanticism,{Romanticism:200},NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2476,Mahmud Taghiyev,Azerbaijani,Baku,1923.0,"Expressionism, Realism","{Expressionism:16},{Realism:1}",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2483,Dominique Gonzalez-Foerster,NaN,Strasbourg,1965.0,"Conceptual Art, Unknown","{Conceptual Art:10},{Unknown:1}",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2533,Carlos Quizpez Asín,NaN,Lima,1900.0,"Cubism, Expressionism, Muralism, Neoclassicism","{Cubism:3},{Expressionism:9},{Muralism:6},{Neo...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2575,Ivan Mrkviсka,NaN,NaN,NaN,"Impressionism, Orientalism, Post-Impressionism...","{Impressionism:35},{Orientalism:1},{Post-Impre...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2615,William Hawkins,NaN,Vance County,1777.0,"Naïve Art (Primitivism), Outsider art","{Naïve Art (Primitivism):31},{Outsider art:7}",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2617,Felix Gonzalez-Torres,NaN,Guáimaro,1957.0,"Conceptual Art, Minimalism, Unknown","{Conceptual Art:9},{Minimalism:9},{Unknown:1}",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
artists_c[~(artists_c['artist'].isin(list(artists['artist'])))]

,artist,Nationality,birth_place,birth_year,styles,styles_extended,StylesYears,StylesCount,PlacesCount,Contemporary,...,FirstYear,LastYear,Places,PlacesYears,PaintingSchool,Influencedby,Influencedon,Pupils,Teachers,FriendsandCoworkers
160,Michael Bell,NaN,County Louth,1936.0,New Realism,{New Realism:9},NaN,NaN,NaN,Yes,...,2010.0,2010.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
163,Arthur Pan,NaN,NaN,NaN,Realism,{Realism:8},NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
167,Georges Troubat,NaN,NaN,NaN,Abstract Art,{Abstract Art:12},NaN,NaN,{France:70},No,...,1480.0,1490.0,France,France:1480-1490,NaN,NaN,NaN,NaN,NaN,NaN


## Version 2024.01.19 (updates 2024.01.16): Take the intersection of WikiArt and Art500k, look for similar names in Art500k and add them too

In [5]:
artists= artists_wikiart[artists_wikiart['artist'].isin(art500k_artists['artist'])].reset_index(drop=True)
drop = artists_wikiart[~(artists_wikiart['artist'].isin(art500k_artists['artist']))]
print("Artists remaining:", len(artists), "\n", "Artists dropped:", len(drop))

Artists remaining: 2458 
 Artists dropped: 745


Let's try to find painters with some name very similar to the ones in WikiArt:

In [6]:
import difflib
#from fuzzywuzzy import fuzz #Other possibility

# Function to calculate similarity between two strings
def similarity(s1, s2):
    return difflib.SequenceMatcher(None, s1, s2).ratio()

def similarity_difference(s1, s2):
    return (1 - similarity(s1, s2))*len(s1)

In [7]:
def levenshtein_distance(s1, s2):
    if len(s1) < len(s2):
        return levenshtein_distance(s2, s1)

    if len(s2) == 0:
        return len(s1)

    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    
    return previous_row[-1]

In [ ]:
#Could use numpy to be faster, but this is fine for now
drop_sims = pd.DataFrame(columns=['artist','"Best" Art500k pair','similarity', 'Character difference'])
for painter in drop['artist']:
    all_sims = []
    max_sim = 0
    for art500k_artist in art500k_artists['artist']:
        similarity_score = similarity(painter, art500k_artist)
        if similarity_score >= max_sim: #Runtime reasons
            max_sim = similarity_score
            all_sims.append((similarity_score,(1-similarity_score)*len(painter), art500k_artist))
    final_maximum = max(sims[0] for sims in all_sims) 
    for sims in all_sims:
        if sims[0] == final_maximum: #Just take the highest ones
            drop_sims = pd.concat([drop_sims, pd.DataFrame([[painter, sims[2], sims[0], sims[1]]], columns=['artist','"Best" Art500k pair','similarity', 'Character difference'])])    
#drop_sims.sort_values(by=['similarity'], ascending=False)

In [129]:
drop_sims = drop_sims.reset_index(drop=True)
drop_sims[drop_sims['Character difference'] < 1.01 ].sort_values(by=['similarity'], ascending=False)[-15:]

,artist,"""Best"" Art500k pair",similarity,Character difference
273,Ayse Erkmen,Ayşe Erkmen,0.909091,1.000000
588,Park Seo-Bo,Park Seo Bo,0.909091,1.000000
587,Park Seo-Bo,Park Seo-bo,0.909091,1.000000
191,Zao Wou-Ki,Zao Wou Ki,0.900000,1.000000
225,Léo Schnug,Leo Schnug,0.900000,1.000000
558,Tony DeLap,Tony Delap,0.900000,1.000000
447,SM Sultan,Sm Sultan,0.888889,1.000000
70,Jay DeFeo,Jay Defeo,0.888889,1.000000
189,Se-Ok Suh,Se Ok Suh,0.888889,1.000000
606,Ay-O,Ay O,0.750000,1.000000


In [ ]:
painter_name_pairs_dict = {}
art500k_alias_groups = {}
subset = drop_sims[drop_sims['Character difference'] < 1.01 ].sort_values(by=['similarity'], ascending=False)[:-5].reset_index(drop=True)
for index, row in subset.iterrows():
    painter = row['artist']
    if painter not in painter_name_pairs_dict.keys():
        painter_name_pairs_dict[painter] = subset.loc[index, '"Best" Art500k pair']
        art500k_alias_groups[painter] = [subset.loc[index, '"Best" Art500k pair']]
    else:
        t = art500k_alias_groups[painter]
        art500k_alias_groups[painter] = t + [subset.loc[index, '"Best" Art500k pair']]

In [ ]:
for key, value in art500k_alias_groups.items():
    if len(value) > 1:
        print(key, value)

Juan Carreno de Miranda ['Juan Carreño de Miranda', 'Juan Carreno De Miranda']
Albert Rafols-Casamada ['Albert Ràfols-Casamada', 'Albert Rafols Casamada']
Francisco de Zurbaran ['Francisco De Zurbaran', 'Francisco de Zurbarán']
Andres de Santa Maria ['Andres De Santa Maria', 'Andrés de Santa Maria']
Jean-Honore Fragonard ['Jean Honore Fragonard', 'Jean-Honoré Fragonard']
Theo van Rysselberghe ['Théo van Rysselberghe', 'Theo Van Rysselberghe']
Janos Mattis-Teutsch ['János Mattis-Teutsch', 'Janos Mattis Teutsch']
Edouard Debat-Ponsan ['Édouard Debat-Ponsan', 'Edouard Debat Ponsan']
Juan de Valdes Leal ['Juan de Valdés Leal', 'Juan De Valdes Leal']
Park Seo-Bo ['Park Seo Bo', 'Park Seo-bo']


I now edit these in the Art500k dataset. Let's try again:

In [10]:
art500k_artists = pd.read_csv('https://raw.githubusercontent.com/me9hanics/PainterPalette/main/datasets/art500k_artists.csv')
artists_wikiart = pd.read_csv('https://raw.githubusercontent.com/me9hanics/PainterPalette/main/datasets/wikiart_artists.csv')
artists= artists_wikiart[artists_wikiart['artist'].isin(art500k_artists['artist'])].reset_index(drop=True)
drop = artists_wikiart[~(artists_wikiart['artist'].isin(art500k_artists['artist']))]

In [11]:
drop_sims = pd.DataFrame(columns=['artist','"Best" Art500k pair','similarity', 'Character difference'])
print("Cases:", len(drop))
for painter in drop['artist']:
    if (drop.index[drop['artist'] == painter][0] == len(drop)//4):
        print("25% now drop finding...")
    all_sims = []
    max_sim = 0
    for art500k_artist in art500k_artists['artist']:
        similarity_score = similarity(painter, art500k_artist)
        if similarity_score >= max_sim: #Runtime reasons
            max_sim = similarity_score
            all_sims.append((similarity_score,(1-similarity_score)*len(painter), art500k_artist))
    final_maximum = max(sims[0] for sims in all_sims) 

    for sims in all_sims:
        if sims[0] == final_maximum: #Just take the highest ones
            drop_sims = pd.concat([drop_sims, pd.DataFrame([[painter, sims[2], sims[0], sims[1]]], columns=['artist','"Best" Art500k pair','similarity', 'Character difference'])])    

print("Done with the drop finding...")

painter_name_pairs_dict = {}
art500k_alias_groups = {}
subset = drop_sims[drop_sims['Character difference'] < 1.01 ].sort_values(by=['similarity'], ascending=False)[:-5].reset_index(drop=True)
for index, row in subset.iterrows():
    if (index == len(subset)//4):
        print("25% now...")
    painter = row['artist']
    if painter not in painter_name_pairs_dict.keys():
        painter_name_pairs_dict[painter] = subset.loc[index, '"Best" Art500k pair']
        art500k_alias_groups[painter] = [subset.loc[index, '"Best" Art500k pair']]
    else:
        t = art500k_alias_groups[painter]
        art500k_alias_groups[painter] = t + [subset.loc[index, '"Best" Art500k pair']]


Cases: 745


C:\Users\hanic\AppData\Local\Temp\ipykernel_11588\3101816928.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  drop_sims = pd.concat([drop_sims, pd.DataFrame([[painter, sims[2], sims[0], sims[1]]], columns=['artist','"Best" Art500k pair','similarity', 'Character difference'])])


Done with the drop finding...
25% now...


In [12]:
for key, value in art500k_alias_groups.items():
    if len(value) > 1:
        print(key, value)

So we have fixed succeeded. In the meanwhile, one case now should be dropped:

In [ ]:
del painter_name_pairs_dict['Zaya']

Let's find 2 and more character differences:

In [14]:
drop_sims = drop_sims.reset_index(drop=True)

In [147]:
drop_sims[(drop_sims['Character difference'] > 1.01) & (drop_sims['Character difference'] <2.01) ].sort_values(by=['similarity'], ascending=False)[0:]

,artist,"""Best"" Art500k pair",similarity,Character difference
366,Marevna (Marie Vorobieff),Marevna Marie Vorobieff,0.958333,1.041667
229,Petro Kholodny (Elder),Petro Kholodny Elder,0.952381,1.047619
506,Martín Rico y Ortega,Martín Rico Ortega,0.947368,1.052632
471,"Robert De Niro, Sr.",Robert De Niro Sr,0.944444,1.055556
218,J. C. Leyendecker,J C Leyendecker,0.937500,1.062500
...,...,...,...,...
234,Kim Prisu,Kid Paris,0.777778,2.000000
349,Sun Mu,Sun Xun,0.769231,1.384615
59,Ed Clark,Clark,0.769231,1.846154
456,Alan Lee,Sean Lee,0.750000,2.000000


Manually checked each case

In [16]:
subset = drop_sims[(drop_sims['Character difference'] > 1.01) & (drop_sims['Character difference'] <2.01) ].sort_values(by=['similarity'], ascending=False)[0:30]
for index, row in subset.iterrows():
    painter = row['artist']
    if painter not in painter_name_pairs_dict.keys(): #Cautions, theoretically this should always happen
        painter_name_pairs_dict[painter] = subset.loc[index, '"Best" Art500k pair']
        
painter_name_pairs_dict.update({"Chang Dai-chien": "Chang Dai Chien", "Félix Del Marle":"F Lix Del Marle", "Roger Bissière":"Roger Bissi Re",
                                "Jacques Hérold": "Jacques H Rold", "YiFei Chen": "Yifei Chen", "M.C. Escher": "M C Escher", 
                                "Hong Song-dam": "Hong Song Dam", "Mestre Ataíde": "Mestre Ata De", "Li Yuan-chia":"Li Yuan Chia", "José Luzán": "Jose Luzan"})
del painter_name_pairs_dict['Jacob Collins']
del painter_name_pairs_dict['Michael Bell']                  

More than 2 character differences (looking through all cases):

In [176]:
drop_sims[(drop_sims['Character difference'] > 2.01)].sort_values(by=['similarity'], ascending=False)[630:660]

,artist,"""Best"" Art500k pair",similarity,Character difference
5,Babak-Matveev,Barbara White,0.538462,6.000000
669,Ruth Annaqtuusi Tulurialik,South Australia,0.536585,12.048780
303,EtchingRoom1,Cochino,0.526316,5.684211
276,Haralampi G. Oroschakoff,Jamie Bischoff,0.526316,11.368421
52,[ a y s h ],Mary Lish,0.500000,5.500000
51,[ a y s h ],Cray Fish,0.500000,5.500000
464,Boushra Yahya Almutawakel,Thouraya Hamouda,0.487805,12.804878
498,JAROSLAV KELUC,ALBERT OLIVE,0.461538,7.538462
499,JAROSLAV KELUC,ARCHIVO FIEL,0.461538,7.538462


In [24]:
painter_name_pairs_dict.update({"António de Carvalho da Silva Porto": "Ant Nio De Carvalho Da Silva Porto", "Jean-Joseph-Xavier Bidauld":"Jean-Joseph-Xavier Bidauld (French", "Pieter Saenredam":"Jan Pietersz Saenredam", "Shin Yoon-bok":"Sin Yun-bok",
                                "Jindrich Styrsky": "Jindřich Štyrský", "George Philip Reinagle":"Philip Reinagle", "Ralph Blakelock":"Ralph Albert Blakelock", "Frances Macdonald": "Frances Macdonald Macnair","Pellizza da Volpedo":"Giuseppe Pellizza da Volpedo", "Raimundo de Madrazo":"Raimundo de Madrazo y Garreta",
                                "Elenore Abbott":"Elenore Plaisted Abbott", "C. R. W. Nevinson":"Christopher R. W. Nevinson", "Il Sassetta (Stefano di Giovanni)":"Stefano di Giovanni", "Rafael García Hispaleto (El Hispaleto)":"Rafael García y García. Hispaleto",
                                "Alexei Korzukhin": "Aleksey Ivanovich Korzukhin", "Charles-Andre van Loo (Carle van Loo)":"Charles-André van Loo", "Lubo Kristek":"Lubo Kristek In Landsberg"})

Some upper/lower case difference pairs:

In [179]:
painter_lowercase_pairs = {}
for index, row in drop_sims.iterrows():
    if row['Character difference'] > 1.01:
        painter_lowercase = row['artist'].lower()
        for artist in art500k_artists['artist']:
            if painter_lowercase == artist.lower():
                painter_lowercase_pairs[row['artist']] = artist
                
painter_lowercase_pairs

{'Adam van der Meulen': 'Adam Van Der Meulen',
 'JAROSLAV KELUC': 'Jaroslav Keluc',
 'Ding Yi': 'DING Yi',
 'JCJ Vanderheyden': 'Jcj Vanderheyden',
 'Bart van der Leck': 'Bart Van Der Leck',
 'Luis de Madrazo y Kuntz': 'Luis De Madrazo Y Kuntz',
 'Phase 2': 'PHASE 2',
 'TRACY 168': 'Tracy 168'}

Let's do a safety check: all keys should be in WikiArt, and all values should be in Art500k:

In [27]:
painter_name_pairs_dict.update(painter_lowercase_pairs)
for key, value in painter_name_pairs_dict.items():
    if key not in artists_wikiart['artist'].str.strip().values:
        print(key, "not in artists_wikiart")
    if value not in art500k_artists['artist'].str.strip().values:
        print(value, "not in art500k_artists")

YiFei Chen was in the database with a double space, and Gohar Fermanyan had a space at the end of the name. I fixed these in the WikiArt dataset, reran this, now it's good.

### Combine the two datasets

In [271]:
artists= artists_wikiart[artists_wikiart['artist'].isin(art500k_artists['artist'])].reset_index(drop=True)
artists = artists.merge(art500k_artists, on='artist', how='left')
artists

,artist,styles,movement,styles_extended,pictures_count,birth_place,birth_year,Nationality,PaintingSchool,ArtMovement,...,FriendsandCoworkers,FirstYear,LastYear,Places,PlacesYears,StylesYears,StylesCount,PlacesCount,Contemporary,Type
0,Ad Reinhardt,"Abstract Art, Abstract Expressionism, Color Fi...",Abstract Expressionism,"{Abstract Art:15},{Abstract Expressionism:5},{...",52,Buffalo,1913.0,American,"New York School,American Abstract Artists,Iras...","{Abstract Expressionism,Minimalism:52}",...,"Jackson Pollock,",1937.0,1966.0,"US, NY, Canberra, Fort Worth, Buffalo, Austral...","New York City:1938-1966,NY:1938-1966,US:1938-1...","Expressionism:1944-1946,Abstract Art:1937-1941...","{Expressionism:7}, {Abstract Art:15}, {Color F...","{New York City:29},{NY:31},{US:32},{Buffalo:2}...",No,NaN
1,Adnan Coker,"Abstract Art, Abstract Expressionism",Abstract Art,"{Abstract Art:25},{Abstract Expressionism:3}",28,NaN,NaN,Turkish,NaN,{Abstract Art:28},...,NaN,1968.0,2008.0,NaN,NaN,"Abstract Art:1992-2008,Abstract Expressionism:...","{Abstract Art:25}, {Abstract Expressionism:3}",NaN,Yes,NaN
2,Akkitham Narayanan,Abstract Art,Abstract Art,{Abstract Art:17},17,Kerala,1939.0,Indian,NaN,{Abstract Art:17},...,NaN,1974.0,1974.0,NaN,NaN,Abstract Art:1974-1974,{Abstract Art:17},NaN,No,NaN
3,Alberto Magnelli,"Abstract Art, Art Nouveau (Modern), Cubism, Ex...",Abstract Art,"{Abstract Art:19},{Art Nouveau (Modern):2},{Cu...",35,Florence,1888.0,"Italian,French",Abstraction-Création,"{Abstract Art,Cubo-Futurism,Concrete Art (Conc...",...,NaN,1909.0,1971.0,NaN,NaN,"Abstract Art:1916-1971,Cubism:1914-1935,Metaph...","{Abstract Art:21}, {Cubism:10}, {Metaphysical ...",NaN,No,NaN
4,Alekos Kontopoulos,"Abstract Art, Cubism, Expressionism, Post-Impr...",Social Realism,"{Abstract Art:26},{Cubism:5},{Expressionism:10...",79,Lamia,1904.0,Greek,NaN,"{Abstract Art,Social Realism:79}",...,NaN,1931.0,1974.0,NaN,NaN,"Post-Impressionism:1932-1955,Expressionism:193...","{Post-Impressionism:8}, {Expressionism:11}, {R...",NaN,No,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2453,Marianne von Werefkin,Unknown,Expressionism,{Unknown:61},61,Tula,1860.0,NaN,NaN,{Der Blaue Reiter:1},...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2454,Robert Demachy,Unknown,Pictorialism,{Unknown:24},24,Saint-Germain-en-Laye,1859.0,French,NaN,{Pictorialism:24},...,NaN,1900.0,1914.0,France,NaN,NaN,NaN,{France:2},No,NaN
2455,Wolfgang Tillmans,Unknown,Contemporary,{Unknown:9},9,Remscheid,1968.0,NaN,NaN,NaN,...,NaN,2001.0,2001.0,"London, United Kingdom",NaN,NaN,NaN,"{London:1},{United Kingdom:1}",Yes,NaN
2456,Wu Daozi,Unknown,Tang Dynasty (618–907),{Unknown:8},8,Chang'an,680.0,Chinese,Four fathers of Chinese painting,{Tang Dynasty (618–907):8},...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [272]:
for key, value in painter_name_pairs_dict.items():
    wikiart_artist_df = artists_wikiart[artists_wikiart['artist'] == key]; art500k_artist_df = art500k_artists[art500k_artists['artist'] == value]
    columns_list_W = artists_wikiart.columns.tolist(); columns_list_A = art500k_artists.columns.tolist()[1:]
    combined_df = pd.concat([wikiart_artist_df[columns_list_W].reset_index(), art500k_artist_df[columns_list_A].reset_index()],  axis=1).drop(columns=['index'])
    artists = pd.concat([artists, combined_df], axis=0).reset_index(drop=True)
artists

,artist,styles,movement,styles_extended,pictures_count,birth_place,birth_year,Nationality,PaintingSchool,ArtMovement,...,FriendsandCoworkers,FirstYear,LastYear,Places,PlacesYears,StylesYears,StylesCount,PlacesCount,Contemporary,Type
0,Ad Reinhardt,"Abstract Art, Abstract Expressionism, Color Fi...",Abstract Expressionism,"{Abstract Art:15},{Abstract Expressionism:5},{...",52,Buffalo,1913.0,American,"New York School,American Abstract Artists,Iras...","{Abstract Expressionism,Minimalism:52}",...,"Jackson Pollock,",1937.0,1966.0,"US, NY, Canberra, Fort Worth, Buffalo, Austral...","New York City:1938-1966,NY:1938-1966,US:1938-1...","Expressionism:1944-1946,Abstract Art:1937-1941...","{Expressionism:7}, {Abstract Art:15}, {Color F...","{New York City:29},{NY:31},{US:32},{Buffalo:2}...",No,NaN
1,Adnan Coker,"Abstract Art, Abstract Expressionism",Abstract Art,"{Abstract Art:25},{Abstract Expressionism:3}",28,NaN,NaN,Turkish,NaN,{Abstract Art:28},...,NaN,1968.0,2008.0,NaN,NaN,"Abstract Art:1992-2008,Abstract Expressionism:...","{Abstract Art:25}, {Abstract Expressionism:3}",NaN,Yes,NaN
2,Akkitham Narayanan,Abstract Art,Abstract Art,{Abstract Art:17},17,Kerala,1939.0,Indian,NaN,{Abstract Art:17},...,NaN,1974.0,1974.0,NaN,NaN,Abstract Art:1974-1974,{Abstract Art:17},NaN,No,NaN
3,Alberto Magnelli,"Abstract Art, Art Nouveau (Modern), Cubism, Ex...",Abstract Art,"{Abstract Art:19},{Art Nouveau (Modern):2},{Cu...",35,Florence,1888.0,"Italian,French",Abstraction-Création,"{Abstract Art,Cubo-Futurism,Concrete Art (Conc...",...,NaN,1909.0,1971.0,NaN,NaN,"Abstract Art:1916-1971,Cubism:1914-1935,Metaph...","{Abstract Art:21}, {Cubism:10}, {Metaphysical ...",NaN,No,NaN
4,Alekos Kontopoulos,"Abstract Art, Cubism, Expressionism, Post-Impr...",Social Realism,"{Abstract Art:26},{Cubism:5},{Expressionism:10...",79,Lamia,1904.0,Greek,NaN,"{Abstract Art,Social Realism:79}",...,NaN,1931.0,1974.0,NaN,NaN,"Post-Impressionism:1932-1955,Expressionism:193...","{Post-Impressionism:8}, {Expressionism:11}, {R...",NaN,No,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2661,Gohar Fermanyan,Post-Impressionism,Post-Impressionism,{Post-Impressionism:3},3,NaN,NaN,Armenian,NaN,{Post-Impressionism:3},...,NaN,NaN,NaN,NaN,NaN,NaN,{Post-Impressionism:3},NaN,NaN,NaN
2662,JAROSLAV KELUC,Impressionism,Impressionism,{Impressionism:33},33,NaN,NaN,Czech,NaN,{Impressionism:33},...,NaN,1949.0,1979.0,NaN,NaN,Impressionism:1949-1979,{Impressionism:33},NaN,No,NaN
2663,Ding Yi,Maximalism,Maximalism,{Maximalism:29},29,"Suixi County, Anhui",150.0,NaN,NaN,NaN,...,NaN,1989.0,1991.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2664,Phase 2,Street art,Street art,{Street art:13},13,NaN,NaN,NaN,NaN,{Street art:2},...,NaN,NaN,NaN,"New York, United States",NaN,NaN,NaN,"{New York:1},{United States:1}",NaN,NaN


In [273]:
cols = artists.columns.tolist(); print(cols)
cols = cols[0:1]+cols[7:8]+cols[5:7]+cols[1:2]+cols[3:4]+cols[19:]+cols[2:3]+cols[9:10]+cols[4:5]+cols[15:19]+cols[8:9]+cols[10:15]
artists = artists[cols]
artists

['artist', 'styles', 'movement', 'styles_extended', 'pictures_count', 'birth_place', 'birth_year', 'Nationality', 'PaintingSchool', 'ArtMovement', 'Influencedby', 'Influencedon', 'Pupils', 'Teachers', 'FriendsandCoworkers', 'FirstYear', 'LastYear', 'Places', 'PlacesYears', 'StylesYears', 'StylesCount', 'PlacesCount', 'Contemporary', 'Type']


,artist,Nationality,birth_place,birth_year,styles,styles_extended,StylesYears,StylesCount,PlacesCount,Contemporary,...,FirstYear,LastYear,Places,PlacesYears,PaintingSchool,Influencedby,Influencedon,Pupils,Teachers,FriendsandCoworkers
0,Ad Reinhardt,American,Buffalo,1913.0,"Abstract Art, Abstract Expressionism, Color Fi...","{Abstract Art:15},{Abstract Expressionism:5},{...","Expressionism:1944-1946,Abstract Art:1937-1941...","{Expressionism:7}, {Abstract Art:15}, {Color F...","{New York City:29},{NY:31},{US:32},{Buffalo:2}...",No,...,1937.0,1966.0,"US, NY, Canberra, Fort Worth, Buffalo, Austral...","New York City:1938-1966,NY:1938-1966,US:1938-1...","New York School,American Abstract Artists,Iras...","Piet Mondrian,Kazimir Malevich,Josef Albers,","Donald Judd,Barnett Newman,Mark Rothko,Frank S...",NaN,NaN,"Jackson Pollock,"
1,Adnan Coker,Turkish,NaN,NaN,"Abstract Art, Abstract Expressionism","{Abstract Art:25},{Abstract Expressionism:3}","Abstract Art:1992-2008,Abstract Expressionism:...","{Abstract Art:25}, {Abstract Expressionism:3}",NaN,Yes,...,1968.0,2008.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Akkitham Narayanan,Indian,Kerala,1939.0,Abstract Art,{Abstract Art:17},Abstract Art:1974-1974,{Abstract Art:17},NaN,No,...,1974.0,1974.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Alberto Magnelli,"Italian,French",Florence,1888.0,"Abstract Art, Art Nouveau (Modern), Cubism, Ex...","{Abstract Art:19},{Art Nouveau (Modern):2},{Cu...","Abstract Art:1916-1971,Cubism:1914-1935,Metaph...","{Abstract Art:21}, {Cubism:10}, {Metaphysical ...",NaN,No,...,1909.0,1971.0,NaN,NaN,Abstraction-Création,NaN,NaN,NaN,NaN,NaN
4,Alekos Kontopoulos,Greek,Lamia,1904.0,"Abstract Art, Cubism, Expressionism, Post-Impr...","{Abstract Art:26},{Cubism:5},{Expressionism:10...","Post-Impressionism:1932-1955,Expressionism:193...","{Post-Impressionism:8}, {Expressionism:11}, {R...",NaN,No,...,1931.0,1974.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2661,Gohar Fermanyan,Armenian,NaN,NaN,Post-Impressionism,{Post-Impressionism:3},NaN,{Post-Impressionism:3},NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2662,JAROSLAV KELUC,Czech,NaN,NaN,Impressionism,{Impressionism:33},Impressionism:1949-1979,{Impressionism:33},NaN,No,...,1949.0,1979.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2663,Ding Yi,NaN,"Suixi County, Anhui",150.0,Maximalism,{Maximalism:29},NaN,NaN,NaN,NaN,...,1989.0,1991.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2664,Phase 2,NaN,NaN,NaN,Street art,{Street art:13},NaN,NaN,"{New York:1},{United States:1}",NaN,...,NaN,NaN,"New York, United States",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [268]:
artists.to_csv('artists.csv', index=False); artists.to_csv('datasets/artists.csv', index=False)

### Re-update 2024.01.28: Take the mapping

In [30]:
for painter in artists_wikiart['artist']:
    if painter in list(art500k_artists['artist']):
        painter_name_pairs_dict[painter] = painter  

In [33]:
#Save the dictionary
import json

with open('datasets/saves/painter_name_pairs.json', 'w') as fp:
    json.dump(painter_name_pairs_dict, fp)

## Version 2023.12.02: Take the intersection of WikiArt and Art500k

In [95]:
artist_A = pd.read_csv('datasets/wikiart_artists.csv')
artists= artist_A[artist_A['artist'].isin(art500k_artists['artist'])].reset_index(drop=True)
print("Artists remaining:", len(artists))

Artists remaining: 2457


In [96]:
artists = artists.merge(art500k_artists, on='artist', how='left')
artists

,artist,styles,movement,styles_extended,pictures_count,birth_place,birth_year,Nationality,PaintingSchool,ArtMovement,...,Influencedon,Pupils,Teachers,FriendsandCoworkers,FirstYear,LastYear,Places,PlacesYears,StylesYears,StylesCount
0,Ad Reinhardt,"Abstract Art, Abstract Expressionism, Color Fi...",Abstract Expressionism,"{Abstract Art:15},{Abstract Expressionism:5},{...",52,Buffalo,1913.0,American,"New York School,American Abstract Artists,Iras...","{Abstract Expressionism,Minimalism:52},",...,"Donald Judd,Barnett Newman,Mark Rothko,Frank S...",NaN,NaN,"Jackson Pollock,",1937.0,1966.0,"US, NY, Canberra, Fort Worth, Buffalo, Austral...","New York City:1938-1966,,NY:1938-1966,,US:1938...","Expressionism:1944-1946,,Abstract Art:1937-194...","{Expressionism:7}, {Abstract Art:15}, {Color F..."
1,Adnan Coker,"Abstract Art, Abstract Expressionism",Abstract Art,"{Abstract Art:25},{Abstract Expressionism:3}",28,NaN,NaN,Turkish,NaN,"{Abstract Art:28},",...,NaN,NaN,NaN,NaN,1968.0,2008.0,NaN,NaN,"Abstract Art:1992-2008,,Abstract Expressionism...","{Abstract Art:25}, {Abstract Expressionism:3}"
2,Akkitham Narayanan,Abstract Art,Abstract Art,{Abstract Art:17},17,Kerala,1939.0,Indian,NaN,"{Abstract Art:17},",...,NaN,NaN,NaN,NaN,1974.0,1974.0,NaN,NaN,"Abstract Art:1974-1974,",{Abstract Art:17}
3,Alberto Magnelli,"Abstract Art, Art Nouveau (Modern), Cubism, Ex...",Abstract Art,"{Abstract Art:19},{Art Nouveau (Modern):2},{Cu...",35,Florence,1888.0,"Italian,French",Abstraction-Création,"{Abstract Art,Cubo-Futurism,Concrete Art (Conc...",...,NaN,NaN,NaN,NaN,1909.0,1971.0,NaN,NaN,"Abstract Art:1916-1971,,Cubism:1914-1935,,Meta...","{Abstract Art:21}, {Cubism:10}, {Metaphysical ..."
4,Alekos Kontopoulos,"Abstract Art, Cubism, Expressionism, Post-Impr...",Social Realism,"{Abstract Art:26},{Cubism:5},{Expressionism:10...",79,Lamia,1904.0,Greek,NaN,"{Abstract Art,Social Realism:79},",...,NaN,NaN,NaN,NaN,1931.0,1974.0,NaN,NaN,"Post-Impressionism:1932-1955,,Expressionism:19...","{Post-Impressionism:8}, {Expressionism:11}, {R..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2452,Marianne von Werefkin,Unknown,Expressionism,{Unknown:61},61,Tula,1860.0,NaN,NaN,"{Der Blaue Reiter:1},",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2453,Robert Demachy,Unknown,Pictorialism,{Unknown:24},24,Saint-Germain-en-Laye,1859.0,French,NaN,"{Pictorialism:24},",...,NaN,NaN,NaN,NaN,1900.0,1914.0,France,NaN,NaN,NaN
2454,Wolfgang Tillmans,Unknown,Contemporary,{Unknown:9},9,Remscheid,1968.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2001.0,2001.0,"London, United Kingdom",NaN,NaN,NaN
2455,Wu Daozi,Unknown,Tang Dynasty (618–907),{Unknown:8},8,Chang'an,680.0,Chinese,Four fathers of Chinese painting,"{Tang Dynasty (618–907):8},",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Later extend this list with skipped artists from both datasets

In [97]:
artist_AnotB = artist_A[~artist_A['artist'].isin(art500k_artists['artist'])].reset_index(drop=True).sort_values(by=['pictures_count'], ascending=False)
artist_AnotB.head(10)

,artist,styles,movement,styles_extended,pictures_count,birth_place,birth_year
0,Alfred Freddy Krupa,"Abstract Art, Abstract Expressionism, Academic...",New Ink Art,"{Abstract Art:1},{Abstract Expressionism:1},{A...",735,Karlovac,1971.0
720,Zdzislaw Beksinski,Surrealism,Magic Realism,{Surrealism:707},707,Sanok,1929.0
737,Oleksandr Aksinin,Unknown,Soviet Nonconformist Art,{Unknown:480},480,Kiev,1930.0
140,M.C. Escher,"Art Deco, Art Nouveau (Modern), Cubism, Expres...",Surrealism,"{Art Deco:1},{Art Nouveau (Modern):1},{Cubism:...",470,Leeuwarden,1898.0
121,Oleg Holosiy,"Academicism, Cubism, Expressionism, Naïve Art ...",Neo-Expressionism,"{Academicism:1},{Cubism:5},{Expressionism:30},...",372,Dnipro,1965.0
308,Alexander Roitburd,"Cubism, Transavantgarde",Transavantgarde,"{Cubism:1},{Transavantgarde:263}",264,Odesa,1961.0
377,Maria Bozoky,"Expressionism, Impressionism",Expressionism,"{Expressionism:252},{Impressionism:4}",256,Oradea,1909.0
606,Konstantin Gorbatov,Post-Impressionism,Post-Impressionism,{Post-Impressionism:254},254,Tolyatti,1876.0
590,Felix Nadar,Pictorialism,Pictorialism,{Pictorialism:245},245,rue Saint-Honoré,1820.0
436,J.M.W. Turner,"Impressionism, Romanticism, Unknown",Romanticism,"{Impressionism:1},{Romanticism:243},{Unknown:1}",245,London,1775.0


In [98]:
cols = artists.columns.tolist()
cols

['artist',
 'styles',
 'movement',
 'styles_extended',
 'pictures_count',
 'birth_place',
 'birth_year',
 'Nationality',
 'PaintingSchool',
 'ArtMovement',
 'Influencedby',
 'Influencedon',
 'Pupils',
 'Teachers',
 'FriendsandCoworkers',
 'FirstYear',
 'LastYear',
 'Places',
 'PlacesYears',
 'StylesYears',
 'StylesCount']

In [99]:
cols = cols[0:1]+cols[7:8]+cols[5:7]+cols[1:2]+cols[3:4]+cols[19:]+cols[2:3]+cols[9:10]+cols[4:5]+cols[15:19]+cols[8:9]+cols[10:15]
artists = artists[cols]
artists

,artist,Nationality,birth_place,birth_year,styles,styles_extended,StylesYears,StylesCount,movement,ArtMovement,...,FirstYear,LastYear,Places,PlacesYears,PaintingSchool,Influencedby,Influencedon,Pupils,Teachers,FriendsandCoworkers
0,Ad Reinhardt,American,Buffalo,1913.0,"Abstract Art, Abstract Expressionism, Color Fi...","{Abstract Art:15},{Abstract Expressionism:5},{...","Expressionism:1944-1946,,Abstract Art:1937-194...","{Expressionism:7}, {Abstract Art:15}, {Color F...",Abstract Expressionism,"{Abstract Expressionism,Minimalism:52},",...,1937.0,1966.0,"US, NY, Canberra, Fort Worth, Buffalo, Austral...","New York City:1938-1966,,NY:1938-1966,,US:1938...","New York School,American Abstract Artists,Iras...","Piet Mondrian,Kazimir Malevich,Josef Albers,","Donald Judd,Barnett Newman,Mark Rothko,Frank S...",NaN,NaN,"Jackson Pollock,"
1,Adnan Coker,Turkish,NaN,NaN,"Abstract Art, Abstract Expressionism","{Abstract Art:25},{Abstract Expressionism:3}","Abstract Art:1992-2008,,Abstract Expressionism...","{Abstract Art:25}, {Abstract Expressionism:3}",Abstract Art,"{Abstract Art:28},",...,1968.0,2008.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Akkitham Narayanan,Indian,Kerala,1939.0,Abstract Art,{Abstract Art:17},"Abstract Art:1974-1974,",{Abstract Art:17},Abstract Art,"{Abstract Art:17},",...,1974.0,1974.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Alberto Magnelli,"Italian,French",Florence,1888.0,"Abstract Art, Art Nouveau (Modern), Cubism, Ex...","{Abstract Art:19},{Art Nouveau (Modern):2},{Cu...","Abstract Art:1916-1971,,Cubism:1914-1935,,Meta...","{Abstract Art:21}, {Cubism:10}, {Metaphysical ...",Abstract Art,"{Abstract Art,Cubo-Futurism,Concrete Art (Conc...",...,1909.0,1971.0,NaN,NaN,Abstraction-Création,NaN,NaN,NaN,NaN,NaN
4,Alekos Kontopoulos,Greek,Lamia,1904.0,"Abstract Art, Cubism, Expressionism, Post-Impr...","{Abstract Art:26},{Cubism:5},{Expressionism:10...","Post-Impressionism:1932-1955,,Expressionism:19...","{Post-Impressionism:8}, {Expressionism:11}, {R...",Social Realism,"{Abstract Art,Social Realism:79},",...,1931.0,1974.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2452,Marianne von Werefkin,NaN,Tula,1860.0,Unknown,{Unknown:61},NaN,NaN,Expressionism,"{Der Blaue Reiter:1},",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2453,Robert Demachy,French,Saint-Germain-en-Laye,1859.0,Unknown,{Unknown:24},NaN,NaN,Pictorialism,"{Pictorialism:24},",...,1900.0,1914.0,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2454,Wolfgang Tillmans,NaN,Remscheid,1968.0,Unknown,{Unknown:9},NaN,NaN,Contemporary,NaN,...,2001.0,2001.0,"London, United Kingdom",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2455,Wu Daozi,Chinese,Chang'an,680.0,Unknown,{Unknown:8},NaN,NaN,Tang Dynasty (618–907),"{Tang Dynasty (618–907):8},",...,NaN,NaN,NaN,NaN,Four fathers of Chinese painting,NaN,NaN,NaN,NaN,NaN


In [100]:
artists.to_csv('datasets/artists.csv', index=False)

In [125]:
artists = pd.read_csv('datasets/artists.csv')

### Years modification

In [ ]:
year_mistake = []
for artist in artists['artist']:
    if (artists[artists['artist'] == artist]['LastYear'].iloc[0]-artists[artists['artist'] == artist]['FirstYear'].iloc[0])>90:
        year_mistake.append(artist)
print((year_mistake))

In [ ]:
artists[artists['artist'].isin(year_mistake)][['artist','birth_year','FirstYear','LastYear']]

In [129]:
too_early_years = ["Huang Yongyu","Joe Goode","Theodoros Stamos","Pablo Picasso", "Modest Cuixart","Giovanni Paolo Panini", "Guido Reni", "John Riley", "Marcello Bacciarelli","Rembrandt","Alfredo Volpi", "Henry Ossawa Tanner", "Pierre Soulages","Hieronymus Bosch","Agnes Lawrence Pelton","George Morland", "Jean-Baptiste Carpeaux"]
too_latest_years = ["Rupert Bunny", "Vasily Polenov", "Giovanni Paolo Panini", "Guido Reni","John Riley", "Luca Giordano", "Matthias Stom","Rembrandt", "Giovanni Bellini", "Alfredo Volpi", "Francesco Melzi", "Auguste Rodin", "Edgar Degas", "Henry Ossawa Tanner", "John Frederick Kensett","Giorgio de Chirico", "Maria Sibylla Merian", "Hieronymus Bosch","Jan Provoost","Jean Fouquet","Anton Azbe", "Jean-Baptiste Carpeaux"]
second_batch=['Hieronymus Bosch',
 'Jan Provoost',
 'George Lambert',
 'Charles Turner',
 'Thomas Jones',
 'William Morris']


In [128]:
for artist in too_early_years:
    artists.loc[artists['artist'] == artist, 'FirstYear'] = artists[artists['artist'] == artist]['birth_year']+18
#The latest_years artists are manually corrected.

In [130]:
#Manual edit last years
their_last_year = [1947, 1898, 1765, 1642, 1641, 1705, 1649, 1669, 1516, 1988, 1570, 1917, 1917, 1937, 1872, 1978, 1705, 1705, 1705, 1529, 1460, 1900, 1875]
last_years = [1516, 1460, 1802, 1832, 1803, 1892]
for i in range(len(too_latest_years)):
    artists.loc[artists['artist'] == too_latest_years[i], 'LastYear'] = their_last_year[i]
for i in range(len(second_batch)):
    artists.loc[artists['artist'] == second_batch[i], 'LastYear'] = last_years[i]

In [ ]:
artists = artists.merge(subset, on='artist', how='left')

In [145]:

cols = artists.columns.to_list()
cols  = cols[0:15]+cols[-1:]+cols[15:-1]
cols.remove('PlacesCount_x')
artists = artists[cols]
artists.rename(columns={'PlacesCount_y':'PlacesCount'}, inplace=True)
artists.columns

Index(['artist', 'Nationality', 'birth_place', 'birth_year', 'styles',
       'styles_extended', 'StylesYears', 'StylesCount', 'movement',
       'ArtMovement', 'pictures_count', 'FirstYear', 'LastYear', 'Places',
       'PlacesYears', 'PlacesCount', 'PaintingSchool', 'Influencedby',
       'Influencedon', 'Pupils', 'Teachers', 'FriendsandCoworkers'],
      dtype='object')

Last step: in the .csv file, replace float .0 values with integers<br>
*This cannot be precisely done in Pandas, as you cannot have an integer datatype column (Series) with NaNs.*


In [63]:
#Turn the non-NaN years into integers
#t1 = artists['FirstYear'].fillna(0).astype(int).replace(0, "remove_hrgldg")
#t2 = artists['LastYear'].fillna(0).astype(int).replace(0, "remove_hrgldg")
#t3 = artists['birth_year'].fillna(0).astype(int).replace(0, "remove_hrgldg")
#
#artists['FirstYear'] = t1
#artists['LastYear'] = t2
#artists['birth_year'] = t3
#
#artists.to_csv('datasets/artists.csv', index=False)
#Manually delete the cells with "remove_hrgldg"

NOTE: manually deleted the cells containing "remove_hrgldg" from the csv file.

In [64]:
artists = pd.read_csv('datasets/artists.csv')